In [1]:
import os
import time 
import scipy
import numpy as np 
import matplotlib 
import matplotlib.pyplot as plt 
plt.style.use('./utils/tecplot.mplstyle')

import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import torch.utils as utils 
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# import neuralop 
# from utilities import *
from utils.neuraloperator import * 

from pathlib import Path
from torch import Tensor
from typing import Any, List, Tuple, Mapping, Optional, Iterable, Union, Dict, Literal

## libraries for CFD
# import cupy as cp
import h5py 
import yaml

from tqdm import tqdm
import wandb

cuda


In [ ]:
config={
    'name': '2dHIT_FNOKernel_nu=0.000225_T=0.25_fDNS=64', 
    'model': {
        'name': '2dHIT_FNOKernel_nu=0.000225_T=0.25_fDNS=64',
        'params': {
            'n_modes': (32, 32), 
            'in_channels': 1, 
            'out_channels': 1, 
            'hidden_channels': 64, 
            'lifting_channel_ratio': 2, 
            'projection_channel_ratio': 8, 

            # 'factorization':'tucker',
            # 'implementation':'factorized',
            # 'rank': 2, 
            'n_layers': 4,
            'norm': 'instance_norm', # (None, 'instance_norm', 'group_norm', 'ada_in'); 'ada_in'은 에러남 
            # 'fno_skip': 'linear', # ('linear', 'soft-gating', 'identity')
            # 'channel_mlp_skip': 'linear', # ('soft-gating', 'linear', 'identity')
            'positional_embedding': None, # (None, 'grid', GridEmbedding2D, GridEmbeddingnD)
            # 'implementation': 'factorized', # ('factorized', 'reconstructed')
            # 'fft_norm': 'forward', # ('forward', 'ortho', 'backward')
            # 'fno_block_precision': 'full', # ('full', 'mixed', 'half')
            'use_channel_mlp': False, 

            # 'SpectralConv_initializer': 'zeros', # ('normal', 'uniform', 'constant', 'ones', 'zeros', 'eye', 'dirac', 'xavier_uniform', 'xavier_normal', 'kaiming_uniform', 'kaiming_normal', 'trunc_normal', 'orthogonal', 'sparse')
            # # 'SpectralConv_initializer_param': 0.25, # (None, ) 
            # 'SpectralConv_non_linearity': 'gelu', # ('gelu', 'relu', 'silu', 'tanh', 'sigmoid', 'leakyrelu') 

            'use_cnn_skip': False, 
            # 'rm_fno_out': False, 
            # 'cnn_n_layers': 2,
            # 'cnn_hidden_channels': 128,
            # 'cnn_scale_mode': 'stationary_learning',
            # 'cnn_scale_params': {'d': 2, 'alpha': 0.5},

            'SpectralConv_use_cnn_skip': True,
            'SpectralConv_cnn_n_layers': 4,
            # 'SpectralConv_cnn_hidden_channels': 64,
            'SpectralConv_cnn_hidden_channel_ratio': 4,
            'SpectralConv_rm_low_pass_filter': False,
            'SpectralConv_rm_fno_out': False,
            'SpectralConv_rm_lpf_after_conv': False,
            'SpectralConv_cnn_concat': True,
            'Spectral_attention': None, # (None, 'SELayer', 'CBAM', 'BAM', 'SRM', 'GCModule', 'ECALayer', 'GCT')
            # 'SpectralConv_cnn_scale_mode': 'cnn', # 'stationary_learning',
            # 'SpectralConv_cnn_scale_params': {'d': 2, 'alpha': 0.5}, 
        
        },
        'device': device,
    },
    
    'epochs': 50, 
    'optimizer': {
        'loss_fn': 'h1',
        'name': 'Adam',
        'params': {
            'lr': 1e-3,
        },

        'scheduler': {
            'name': 'CosineAnnealingLR', 
        },
    },
    'data': [
        # { 'nu': 0.001, 'leadtime': 0.25, # 'idx_leadtime': 1, 
        #  'stage': 'fit',
        # 'params': {
        #     'base_path': r'D:\RESEARCH\2DIso\Data\nu=0.001_n=128_fDNS=64/', 
        #     'dataset_name': f'2dHIT_nu=0.001_n=128_T=11.5_fDNS=64', 
            
        #     'n_input': 1, 
        #     'n_output':1, 
        #     'batch_size': 64,
        #     'num_workers': 0, 

        #     'Ndata_train':1000,  
        #     'Ndata_val':50, 
        # },
        # 'normalization': {
        #     'normalization_path': r'D:\RESEARCH\2DIso\Data\nu=0.001_n=128_fDNS=64/' + 'config.yaml',
        # }
        # },
       { 'nu': 0.000225, 'leadtime': 0.25, # 'idx_leadtime': 1, 
         'stage': 'fit',
        'params': {
            'base_path': r'D:\RESEARCH\2DIso\Data\nu=0.00025_n=256_fDNS=64/', 
            'dataset_name': f'2dHIT_nu=0.000225_n=256_T=14.5_fDNS=64', 
            
            'n_input': 1, 
            'n_output':1, 
            'batch_size': 64,
            'num_workers': 0, 

            'Ndata_train':1000,  
            'Ndata_val':50, 
        },
        'normalization': {
            'normalization_path': r'D:\RESEARCH\2DIso\Data\nu=0.00025_n=256_fDNS=64/' + 'config.yaml',
        }
        },
        # { 'nu': 5e-04, 'leadtime': 0.25, # 'idx_leadtime': 1, 
        #  'stage': 'fit',
        # 'params': {
        #     'base_path': 'D:\RESEARCH/2DIso\Data/nu=0.0005_n=256_fDNS=64/', 
        #     'dataset_name': f'2dHIT_nu=0.0005_n=256_T=12.0_fDNS=64', 
            
        #     'n_input': 1, 
        #     'n_output':1, 
        #     'batch_size': 64,
        #     'num_workers': 0, 

        #     'Ndata_train':500,  
        #     'Ndata_val':50, 
        #     'load_data': False, 
        # },
        # 'normalization': {
        #     'normalization_path': 'D:\RESEARCH/2DIso\Data/nu=0.0005_n=256_fDNS=64/config.yaml',
        # }
        # },
        # { 'nu': 5e-05, 'leadtime': 0.25, # 'idx_leadtime': 1, 
        #  'stage': 'fit',
        # 'params': {
        #     'base_path': '../Data/nu=5e-05_n=512_fDNS=64/', 
        #     'dataset_name': f'2dHIT_nu=5e-05_n=512_T=17.0_fDNS=64', 
            
        #     'n_input': 1, 
        #     'n_output':1, 
        #     'batch_size': 64,
        #     'num_workers': 0, 

        #     'Ndata_train':500,  
        #     'Ndata_val':50, 
        #     'load_data': False, 
        # },
        # 'normalization': {
        #     'normalization_path': '../Data/nu=5e-05_n=512_fDNS=64/' + 'config.yaml',
        # }
        # },

        
        {
         'nu': 0.000225, 'leadtime': 0.25, # 'idx_leadtime': 1, 
         'stage': 'valid',
        'params': {
            'base_path': r'D:\RESEARCH\2DIso\Data\nu=0.00025_n=256/', 
            'dataset_name': f'2dHIT_nu=0.000225_n=256_T=14.5', 
            
            'n_input': 1, 
            'n_output':1, 
            'batch_size': 64,
            'num_workers': 0, 

            'Ndata_val':1, 
            'load_data': False, 
        },
        'normalization': {
            'normalization_path': r'D:\RESEARCH\2DIso\Data\nu=0.00025_n=256/' + 'config.yaml',
        }
        },
        
        # {
        #  'nu': 5e-04, 'leadtime': 0.25, # 'idx_leadtime': 1, 
        #  'stage': 'valid',
        # 'params': {
        #     'base_path': 'D:\RESEARCH/2DIso\Data/nu=0.0005_n=256/', 
        #     'dataset_name': f'2dHIT_nu=0.0005_n=256_T=12.0', 
            
        #     'n_input': 1, 
        #     'n_output':1, 
        #     'batch_size': 64,
        #     'num_workers': 0, 

        #     'Ndata_val':5, 
        #     'load_data': False, 
        # },
        # 'normalization': {
        #     'normalization_path': 'D:\RESEARCH/2DIso\Data/nu=0.0005_n=256/config.yaml',
        # }
        # },
        # { 'nu': 5e-05, 'leadtime': 0.25, # 'idx_leadtime': 1, 
        #  'stage': 'valid',
        # 'params': {
        #     'base_path': '../Data/nu=5e-05_n=512/', 
        #     'dataset_name': f'2dHIT_nu=5e-05_n=512_T=17.0', 
            
        #     'n_input': 1, 
        #     'n_output':1, 
        #     'batch_size': 16,
        #     'num_workers': 0,

        #     'Ndata_val':1,  
        #     'load_data': False, 
        # },
        # 'normalization': {
        #     'normalization_path': '../Data/nu=5e-05_n=512/' + 'config.yaml',
        # }
        # },
        # { 'nu': 0.001, 'leadtime': 0.25, # 'idx_leadtime': 1, 
        #  'stage': 'valid',
        # 'params': {
        #     'base_path': r'D:\RESEARCH\2DIso\Data\nu=0.001_n=128/', 
        #     'dataset_name': f'2dHIT_nu=0.001_n=128_T=11.5', 
            
        #     'n_input': 1, 
        #     'n_output':1, 
        #     'batch_size': 64,
        #     'num_workers': 0, 

        #     'Ndata_train':1000,  
        #     'Ndata_val':50, 
        #     'load_data': False, 
        # },
        # 'normalization': {
        #     'normalization_path': r'D:\RESEARCH\2DIso\Data\nu=0.001_n=128/' + 'config.yaml',
        # }
        # },

       
    ]
    
}

In [3]:
### CFD equation parameters ###
integral_timescales = {
    1e-3: 1.870162606239319,
    5e-4: 1.5808453559875488,
    0.000225: 1.3786518573760986, 
    1e-4: 1.3038111189,
    5e-05: 1.15829598903656, 
}

In [4]:
from utils.CFDFunction import *
from utils.Losses import *
from utils.Plots import *

In [5]:
import os
import numpy as np 
import torch 
import h5py
import glob
import time
from typing import Optional, Sequence

def high2low_box(
    x: torch.Tensor,
    scale_factors: Union[float, Sequence[float]] = 0.5,
    *,
    keepdim: bool = False,
    dim: int = 2,
    fft_norm: str = "backward",
    last_var_axis: bool = False,
) -> torch.Tensor:
    """Fourier‑domain box‑style scaling (down / up sampling).

    The function supports tensors shaped either as
        • (batch, channels, *spatial_dims)
        • (batch, channels, *spatial_dims, num_var)

    In the second case the last axis (``num_var``) is *not* included in the FFT
    and therefore remains unchanged.
    """

    if isinstance(scale_factors, float):
        scale_factors = [scale_factors] * dim
    scale_factors = list(scale_factors)
    if len(scale_factors) != dim:
        raise ValueError("`scale_factors` length must equal `dim`.")

    if x.ndim < dim + 2:
        raise ValueError(
            f"Input tensor must have at least {dim + 2} dimensions (got {x.ndim})."
        )

    if last_var_axis:
        spatial_axes = list(range(-(dim + 1), -1))
    else:
        spatial_axes = list(range(-dim, 0))
    spatial_shape = [x.shape[i] for i in spatial_axes]

    sf_tensor = torch.tensor(scale_factors, dtype=x.dtype, device=x.device)
    if torch.allclose(sf_tensor, torch.ones_like(sf_tensor)):
        return x.clone()

    Fx = torch.fft.fftn(x, dim=spatial_axes, norm=fft_norm)

    if (sf_tensor < 1).any():
        Fx = torch.fft.fftshift(Fx, dim=spatial_axes)

        if keepdim:
            mask = torch.ones_like(Fx, dtype=torch.bool)
            for rel_ax, (n, sf) in enumerate(zip(spatial_shape, scale_factors)):
                trim = int(round((1 - sf) * n / 2))
                if trim == 0:
                    continue
                abs_ax = spatial_axes[rel_ax] % Fx.ndim  # positive index
                low = torch.arange(trim, device=x.device)
                high = torch.arange(n - trim, n, device=x.device)
                mask.index_fill_(abs_ax, low, False)
                mask.index_fill_(abs_ax, high, False)
            Fx = Fx * mask
        else:
            slices = [slice(None)] * Fx.ndim
            for rel_ax, (n, sf) in enumerate(zip(spatial_shape, scale_factors)):
                trim = int(round((1 - sf) * n / 2))
                slices[spatial_axes[rel_ax]] = slice(trim, n - trim)
            Fx = Fx[tuple(slices)]

        Fx = torch.fft.ifftshift(Fx, dim=spatial_axes)

    # ----------------------------- up‑sampling branch -------------------------
    if (sf_tensor > 1).any():
        Fx = torch.fft.fftshift(Fx, dim=spatial_axes)

        # F.pad pads the *last* k dims; build list accordingly
        last_axes = list(range(-len(spatial_axes) - (1 if last_var_axis else 0), 0))
        pads: List[int] = []
        for ax in Reflectiond(last_axes):
            if ax in spatial_axes:
                rel = spatial_axes.index(ax)
                n = spatial_shape[rel]
                sf = scale_factors[rel]
                extra = int(round((sf - 1) * n))
                pads.extend([extra // 2, extra - extra // 2])
            else:  # var axis → no pad
                pads.extend([0, 0])
        Fx = F.pad(Fx, pads, mode="constant", value=0.0)
        Fx = torch.fft.ifftshift(Fx, dim=spatial_axes)

    # --------------------------------------------------- inverse FFT ----------
    x_out = torch.fft.ifftn(Fx, dim=spatial_axes, norm=fft_norm).real

    # ------------------------------ match statistics --------------------------
    def _mom(t: torch.Tensor):
        mean = t.mean(dim=spatial_axes, keepdim=True)
        std = t.std(dim=spatial_axes, unbiased=False, keepdim=True).clamp_min(1e-12)
        return mean, std

    mean_in, std_in = _mom(x)
    mean_out, std_out = _mom(x_out)

    x_out = (x_out - mean_out) / std_out * std_in + mean_in
    return x_out

class HIT2dDataset(Dataset):
    def __init__(self, 
        path: Optional[str] = None,
        base_path: Optional[str] = None,
        dataset_name: Optional[str] = None,
        split_name: Optional[str] = None,

        target_path: Optional[str] = None,
        target_base_path: Optional[str] = None,
        target_dataset_name: Optional[str] = None,
        
        normalization:Optional[callable] = None,
        transform:Optional[callable] = None,

        n_input: int = 1,
        n_output: int = 1,
        n_stride: int = 0,
        max_rollout_steps=100,
        max_n_sim: Optional[int] = None,
        # batch_size:int=32, 
        
        # num_iteration_per_data:int=None, 
        # isDataAugmentation:bool=False,
        load_data:bool=True,
        verbose:bool=True,
        **kwargs, 
        ):
        super().__init__()
        self.path = path
        self.base_path = base_path
        self.dataset_name = dataset_name
        self.split_name = split_name
        if not path: 
            path = os.path.join(self.base_path, self.split_name, self.dataset_name) # f"{self.base_path}/{self.split_name}/{self.dataset_name}*"
        self.path = sorted(glob.glob(f"{path}*"))
        assert self.path, f"Error: Dataset path {path} does not exist."

        self.target_path = self.path
        if target_path is not None or target_base_path is not None:
            self.target_base_path = target_base_path
            self.target_dataset_name = target_dataset_name
            self.target_split_name = split_name
            if not target_path: 
                target_path = os.path.join(self.target_base_path, self.split_name, self.target_dataset_name) # f"{self.base_path}/{self.split_name}/{self.dataset_name}*"
            self.target_path = sorted(glob.glob(f"{target_path}*"))
            assert self.target_path, f"Error: Dataset path {target_path} does not exist."

        self.normalization = normalization
        self.transform = transform

        self.max_rollout_steps = max_rollout_steps
        self.n_input = n_input
        self.n_output = n_output
        self.n_stride = n_stride
        self.max_n_sim = max_n_sim if max_n_sim else np.inf
        
        self.verbose=verbose
        self.kwargs = kwargs

        self._build_metadata()
        self._calc_len()
        self.load_data = load_data
        if load_data:
            self.data = self._load_data(self.path)
            self.target_data = self._load_data(self.target_path) if self.target_path is not None else self.data

    def _calc_len(self):
        self.n_sim = min(self.max_n_sim, sum(self.n_sim_per_file)) # self.n_sim = sum(self.n_sim_per_file) # len(self.data)
        self.n_steps_per_sim = self.Nt
        self.n_windows_per_sim = self.n_steps_per_sim - (self.n_input + self.n_output + self.n_stride) + 1
        self.len = self.n_sim * self.n_windows_per_sim

    def __len__(self):
        return self.len
    
    def __getitem__(self, idx:int):
        data = self._load_one_sample(idx)
        data = self._preprocess_data(data)
        if self.transform:
            data = self.transform(data)
        if self.normalization:
            data = self.normalization(data)      
        data = self._postprocess_data(data)
        return data
    
    def _load_one_sample(self, idx:int):
        isim = idx // self.n_windows_per_sim
        it = idx % self.n_windows_per_sim
        if self.load_data:
            data = self.data[isim, it:it + self.n_input] # (n_in, channels, *datashape)
            target = self.target_data[isim, it + self.n_input + self.n_stride:it + self.n_input + self.n_stride + self.n_output] # (n_out, channels, *datashape)
        else: 
            for i in range(len(self.n_sim_per_file)):
                ifile = i 
                if isim < sum(self.n_sim_per_file[:i+1]): break 
            
            isim = isim - sum(self.n_sim_per_file[:i])
            with h5py.File(self.path[ifile], 'r') as f:
                data = f["fields"]['vorticity'][isim, it:it + self.n_input] # (n_in, channels, *datashape)
                data = torch.from_numpy(data)
                
            with h5py.File(self.target_path[ifile], 'r') as f:
                target = f["fields"]['vorticity'][isim, it + self.n_input + self.n_stride:it + self.n_input + self.n_stride + self.n_output] # (n_out, channels, *datashape)
                target = torch.from_numpy(target)
        return data, target

    def _load_data(self, path=None,):
        if not path: path = self.path
        data = []
        self.n_sim = 0
        if self.verbose: start_time = time.time()
        for p in path:
            with h5py.File(p, 'r') as f:
                _n_sim = f.attrs["n_trajectories"]
                
                end = min(self.max_n_sim - self.n_sim, _n_sim)
                vorticity = f["fields"]['vorticity'][:end]# vorticity = f["fields"]['vorticity'][:]
                if self.verbose: 
                    print(f'Data Loaded from{p}. shape: {vorticity.shape}, memory: {vorticity.nbytes/1e6} (MB)')
            
            data.append(vorticity)
            self.n_sim += len(vorticity)
            if self.max_n_sim <= self.n_sim: break
            
        data = np.concatenate(data, axis=0)
        memory = data.nbytes
        data = torch.from_numpy(data)
        # self.data = self.data.reshape(self.data.size(0) * self.data.size(1), self.data.size()[2:]) # self.data = torch.concat(self.data, dim=0)
        if self.verbose: print(f'Data Loaded. shape: {data.shape}, dtype: {data.dtype}, time: {time.time() - start_time} (sec), memory: {memory/1e6} (MB)')
        self._calc_len()
        return data
        
    def _build_metadata(self):
        
        with h5py.File(self.path[0], 'r') as f:
            self.nu = f['scalars']['nu'][()]

            self.t = t = f["dimensions"]["t"][:]
            self.x = x = f["dimensions"]["x"][:]
            self.y = y = f["dimensions"]["y"][:]
            self.X, self.Y = np.meshgrid(x, y)

            self.Lx = x[-1] - x[0]
            self.Ly = y[-1] - y[0]
            self.dx = x[1] - x[0]
            self.dy = y[1] - y[0]
            self.Nx = len(x)
            self.Ny = len(y)
            self.dt = t[1] - t[0]
            self.Nt = len(t)
            self.T = t[-1]
            self.t0 = t[0]

        self.n_sim_per_file = []
        for path in self.path:
            with h5py.File(path, 'r') as f:
                n_sim = f.attrs["n_trajectories"]
                self.n_sim_per_file.append(n_sim)
                if self.max_n_sim <= sum(self.n_sim_per_file): break
        
        self.metadata = {
            'nu': self.nu,

            'Lx': self.Lx,
            'Ly': self.Ly,
            'dx': self.dx,
            'dy': self.dy,
            'Nx': self.Nx, 
            'Ny': self.Ny,
            'x': self.x,
            'y': self.y,

            'dt': self.dt,
            'Nt': self.Nt,
            'T': self.T,
            't0': self.t0,
            't': self.t,
        }
        if self.verbose:
            print(f"Loaded dataset metadata: {self.metadata.keys()}")
        return self.metadata
    
    def _preprocess_data(self, data):
        data, target = data

        x = [data, target] # x = torch.cat([data, target], dim=0)

        return x
    def _postprocess_data(self, data):
        data, target = data[0], data[1]

        # data = data.flatten(0, 1) # data.reshape(data.shape[0] * data.shape[1], *data.shape[2:]) # (n_in * channels, *datashape)
        # target = target.flatten(0, 1) # target.reshape(target.shape[0] * target.shape[1], *target.shape[2:]) # (n_out * channels, *datashape)

        return data, target

class CustomDataModule:
    def __init__(self, 
                 batch_size:int=32, 
                 Ndata_train:int=500, 
                 Ndata_val:int=50, 
                 Ndata_test:int=100, 
                 num_workers:int=0,
                 transform:Optional[callable]=None,
                 normalization:Optional[callable]=None,
                 *args, **kwargs
                 ):
        self.batch_size = batch_size
        self.Ndata_train = Ndata_train
        self.Ndata_val = Ndata_val
        self.Ndata_test = Ndata_test

        self.num_workers = num_workers

        self.transform = transform
        self.normalization = normalization

        self.args = args
        self.kwargs = kwargs

    
    def setup(self, stage:str=None):
        if stage in ['train', 'fit', None]:
            self.train_dataset = HIT2dDataset(split_name='train', max_n_sim=self.Ndata_train, transform=self.transform, normalization=self.normalization, *self.args, **self.kwargs, )
            print(f'Train dataset: {len(self.train_dataset)}')
            
        if stage in ['valid', 'fit', None]:
            self.val_dataset = HIT2dDataset(split_name='valid', max_n_sim=self.Ndata_val, transform=None, normalization=self.normalization, *self.args, **self.kwargs, )
            print(f'Val dataset: {len(self.val_dataset)}')

        if stage in ['test', None]:
            self.test_dataset = HIT2dDataset(split_name='valid', max_n_sim=self.Ndata_test, transform=None, normalization=self.normalization, *self.args, **self.kwargs, )
            print(f'Test dataset: {len(self.test_dataset)}')
    
    def prepare_data(self):
        pass

    def train_dataloader(self):
        if self.train_dataset is None: self.setup(stage='train')
        return torch.utils.data.DataLoader(self.train_dataset, 
                                           batch_size = self.batch_size, 
                                           shuffle=True, 
                                           num_workers=self.num_workers, 
                                           pin_memory=True,
                                           drop_last=True, 
                                           )
    
    def val_dataloader(self):
        if self.val_dataset is None: self.setup(stage='valid')
        return torch.utils.data.DataLoader(self.val_dataset, 
                                           batch_size = self.batch_size, 
                                           shuffle=False, 
                                           num_workers=self.num_workers,
                                           pin_memory=True,
                                           drop_last=True,
                                           )

    def test_dataloader(self):
        if self.test_dataset is None:self.setup(stage='test')
        test_batch_size = self.batch_size # self.test_dataset.n_windows_per_sim # // 8
        return torch.utils.data.DataLoader(self.test_dataset, 
                                           batch_size = test_batch_size, 
                                           shuffle=False, 
                                           num_workers=self.num_workers, 
                                           pin_memory=True,
                                           drop_last=True,
                                           )


class RandomShift(nn.Module):
    def __init__(self, 
        shifts:Union[float, Sequence[float]]=(0.5, 0.5), 
        dims:Union[int, Sequence[int]]=(-2, -1),
        *args, **kwargs
        ):
        super(RandomShift, self).__init__()
        self.shifts = shifts
        self.dims = dims

    def forward(self, 
        x, 
        shifts:Union[float, Sequence[float]]=None, 
        dims:Union[int, Sequence[int]]=None, 
        ):
        if shifts is None: shifts = self.shifts
        if dims is None: dims = self.dims
        
        return self.RandomShift(x, shifts=self.shifts, dims=self.dims)
    
    def RandomShift(self,
        x, 
        shifts:Union[float, Sequence[float]]=(0.5, 0.5), 
        dims:Union[int, Sequence[int]]=(-2, -1)
        ):
        '''
        Random shift the input tensor along the spatial dimensions. 
        Parameters:
        - x (torch.tensor): input tensor, shape 
        - shifts (float or sequence of float): maximum shift fraction along each dimension. 
            If float, the same shift fraction is applied to all dimensions.
            If sequence of float, the length must be equal to the number of spatial dimensions.
            The shift fraction is relative to the size of the dimension.
        - dims (int or sequence of int): dimensions to apply the shift. 
            If int, the same dimension is applied to all spatial dimensions.
            If sequence of int, the length must be equal to the number of spatial dimensions.
        
        Returns:
        - x (torch.tensor): shifted tensor, shape 
        '''
        if isinstance(shifts, float): shifts = [shifts] * len(dims)
        assert len(shifts) == len(dims), "Length of shifts and dims must be equal to the number of spatial dimensions."

        shifts = [int(np.random.uniform(-s, s) * x.shape[d]) for s, d in zip(shifts, dims)]
        x = torch.roll(x, shifts=shifts, dims=dims)
        return x

class Reflection(nn.Module):
    def __init__(self, 
        p: float=0.5,
        dims:Union[int, Sequence[int]]=(-2, -1),
        *args, **kwargs
        ):
        super().__init__()
        self.p = p
        self.dims = dims

    def forward(self, 
        x, 
        p = None, 
        dims:Union[int, Sequence[int]]=None, 
        ):
        if dims is None: dims = self.dims
        if p is None: p = self.p
        
        return self.Reflection(x, p=p, dims=self.dims)
    
    def Reflection(self,
        x, 
        p: float=0.5, 
        dims:Union[int, Sequence[int]]=(-2, -1)
        ):
        '''
        Reflection the input tensor along the spatial dimensions. 
        Parameters:
        - x (torch.tensor): input tensor, shape 
        - dims (int or sequence of int): dimensions to apply the Reflection. 
            If int, the same dimension is applied to all spatial dimensions.
            If sequence of int, the length must be equal to the number of spatial dimensions.
        
        Returns:
        - x (torch.tensor): Reflection tensor, shape 
        '''
        if isinstance(dims, int): dims = [dims]
        
        for d in dims:
            if np.random.rand() < p: x = torch.flip(x, dims=(d,))
        return x

class Reverse(nn.Module):
    def __init__(self, 
        p: float=0.5,
        *args, **kwargs
        ):
        super().__init__()
        self.p = p

    def forward(self, 
        x, 
        p = None, 
        ):
        if p is None: p = self.p
        
        return self.Reverse(x, p=p)
    
    def Reverse(self,
        x, 
        p: float=0.5, 
        ):
        '''
        Reverse the input tensor along the time dimension. 
        Parameters:
        - x (torch.tensor): input tensor, shape 
        - dim (int): dimension to apply the Reverse. 
        
        Returns:
        - x (torch.tensor): Reverse tensor, shape 
        '''
        if np.random.rand() < p: x = -x 
        return x
    
class CustomTransform(nn.Module):
    def __init__(self, 
                *args, **kwargs, 
                ):
        super(CustomTransform, self).__init__()
        # self.file_dir = file_dir
        # self.Ndata = Ndata
        self.kwargs = kwargs 
        self.build_transform()

    def build_transform(self):
        self.transforms = nn.Sequential(
            RandomShift(shifts=0.5, dims=(-2, -1)),
            Reflection(p=0.5, dims=(-2, -1)),
            Reverse(p=0.5),
        )
        return self.transforms
    
    def forward(self, x):
        x = self.transform(x)
        return x
    
    def transform(self, x):
        for t in self.transforms:
            x = t(x)
        return x

    def inverse_transform(self, x):
        for t in self.transforms[::-1]:
            x = t.inverse_transform(x)
        return x

class Standardize(nn.Module):
    def __init__(self, 
        mean:Optional[float]=None, std:Optional[float]=None, 
        normalization_path:Optional[str]=None,

        base_path: Optional[str] = None,
        dataset_name: Optional[str] = None,
        split_name: Optional[str] = None,
        *args, **kwargs, 
        ):
        super(Standardize, self).__init__()
        self.args = args
        self.kwargs = kwargs

        ## build_transform
        if mean is not None and std is not None: 
            self.mean = mean
            self.std = std
        elif normalization_path is not None:
            self.load_stats(normalization_path)
        elif dataset_name is not None: 
            path = os.path.join(base_path, split_name, dataset_name)
            self.calc_stats_from_dataset(path)
        else:
            assert False, "Error: must provide either (mean, std), normalization_path, or (base_path, dataset_name, split_name) to calculate mean and std."
    
    def load_stats(self, path=None):
        assert os.path.exists(path), f"Error: normalization path {path} does not exist."
        with open(path, "r") as f:
            stats = yaml.safe_load(f)['statistics']

            self.mean = stats['mean']
            self.std = stats['std']
        return self.mean, self.std
    
    def calc_stats_from_dataset(self, path=None):
        dataset = HIT2dDataset(path=path, load_data=True, *self.args, **self.kwargs).load_data()
        self.mean = dataset.data.mean().item()
        self.std = dataset.data.std().item()
        return self.mean, self.std

    def forward(self, x):
        data, target = x
        data = self.normalize(data, params=None)[0]
        target = self.normalize(target, params=None)[0]
        return [data, target]
    
    def normalize(self, inpt: Any, params: Dict[str, Any]):
        return (inpt - self.mean) / self.std

    def denormalize(self, inpt: Any, params: Dict[str, Any]):
        return inpt * self.std + self.mean

In [6]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F

from utils.CFDFunction import calc_energy_spectrum
from utils.Plots import plot_spectrum

class Hook(nn.Module):
    def __init__(self, hook_every_n_iter:int=1):
        super(Hook, self).__init__()
        self.iteration = 0
        self.data = {}
        self.data['spectrum'] = {}
        self.hook_every_n_iter = hook_every_n_iter
        self.prefix = ''

    def open_hook(self, prefix:str=''):
        # if self.iteration % self.hook_every_n_iter != 0: return 
        self.data = {}
        self.data['spectrum'] = {}
        self.prefix = prefix

    def close_hook(self, fname:str=''):
        # if self.iteration % self.hook_every_n_iter != 0: return 
        spectrums = list(self.data['spectrum'].values())
        title = list(self.data['spectrum'].keys())

        plot_spectrum(spectrum1=spectrums[0], spectrum2=spectrums[1:], 
                      figsize=(min(len(spectrums), 5)*6, (len(spectrums) // 5 + 1 + int(len(spectrums) % 5 == 0))*6),
                      suptitle=f'Spectrum at iteration {self.iteration} - {self.prefix}',
                      titles=title, overlapping=False, 
                      )
        plt.savefig(fname+'.png', dpi=300)
        plt.close('all')
    
    def set_prefix(self, name:str=''):
        self.prefix = name

    def step(self):
        self.iteration += 1

    @torch.no_grad()
    def __call__(self, x, name:str='', model:nn.Module=None):
        # if self.iteration % self.hook_every_n_iter != 0: return 
        
        ## spectrum
        _spectrum = calc_energy_spectrum(x, dim=(-2, -1), channel_dim=1).sum(dim=0)
        if self.prefix: 
            name = f'{self.prefix}_{name}'
        self.data['spectrum'][name] = _spectrum.detach().cpu().numpy()

hook = Hook(hook_every_n_iter=500)

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, Tuple, Dict, Any, Literal, Union, Iterable

from utils.neuraloperator import SpectralConv as BaseSpectralConv
from utils.neuraloperator import FNOBlocks as BaseFNOBlocks
from utils.neuraloperator import FNO as BaseFNO

from utils.refer.WTConv_main.wtconv.wtconv2d import WTConv2d

class CNO_activation(nn.Module):
    def __init__(self, 
        out_shape: Optional[tuple] = None,
        activation: callable = F.gelu,
        ):
        super().__init__()
        self.out_shape = out_shape
        self.act = activation

    def forward(self, x: torch.Tensor, out_shape:Optional[tuple]=None) -> torch.Tensor:
        in_size = [2 * s for s in x.shape[2:]]
        out_shape = list(x.shape[2:]) if out_shape is None and self.out_shape is None else self.out_shape if out_shape is None else out_shape

        x = F.interpolate(x, size=in_size, mode='bilinear', align_corners=True)
        x = self.act(x)
        x = F.interpolate(x, size=out_shape, mode='bilinear', align_corners=True)
        return x
    
class Dealiasing_activation(nn.Module):
    def __init__(self, 
        scale: float = 3/2,
        activation: callable = F.gelu,
        ):
        super().__init__()
        self.scale = scale
        self.act = activation
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.dealiasing(x, scale=self.scale, fn=self.act)
        return x

    def dealiasing(self, x: torch.Tensor, 
                scale: float = 3/2, 
                fn=None):
        dim = list(np.arange(2, len(x.shape)))
        # original spatial size
        orig_size = [x.size(d) for d in dim]
        new_size = [int(s * scale) for s in orig_size]
        idx = [slice(None), slice(None)]
        for n, o in zip(new_size, orig_size):
            start = (n - o) // 2
            end = start + o
            idx.append(slice(start, end))
        new_size = list(x.shape[:2]) + new_size

        # (1) FFT in low-res
        Fx = torch.fft.fftn(x, dim=dim)
        Fx = torch.fft.fftshift(Fx, dim=dim)
        Fx_pad = torch.zeros(new_size, dtype=Fx.dtype, device=x.device)
        Fx_pad[idx] = Fx

        x_pad = torch.fft.ifftn(Fx_pad, dim=dim).real

        x_pad = fn(x_pad)

        Fx_pad = torch.fft.fftn(x_pad, dim=dim)
        Fx_pad = torch.fft.fftshift(Fx_pad, dim=dim)

        Fx = Fx_pad[idx]
        Fx = torch.fft.fftshift(Fx, dim=dim)
        x = torch.fft.ifftn(Fx, dim=dim).real  # often want real

        return x

    

def init_t_xy(end_x: int, end_y: int):
    t = torch.arange(end_x * end_y, dtype=torch.float32)
    t_x = (t % end_x).float()
    t_y = torch.div(t, end_x, rounding_mode='floor').float()
    return t_x, t_y

def compute_mixed_cis(freqs: torch.Tensor, t_x: torch.Tensor, t_y: torch.Tensor, num_heads: int):
    N = t_x.shape[0]
    # No float 16 for this range
    with torch.cuda.amp.autocast(enabled=False):
        freqs_x = (t_x.unsqueeze(-1) @ freqs[0].unsqueeze(-2)).view(N, num_heads, -1).permute(1, 0, 2)
        freqs_y = (t_y.unsqueeze(-1) @ freqs[1].unsqueeze(-2)).view(N, num_heads, -1).permute(1, 0, 2)
        freqs_cis = torch.polar(torch.ones_like(freqs_x), freqs_x + freqs_y)
    return freqs_cis

def init_2d_freqs(dim: int, num_heads: int, theta: float = 10.0, rotate: bool = True):
    freqs_x = []
    freqs_y = []
    mag = 1 / (theta ** (torch.arange(0, dim, 4)[: (dim // 4)].float() / dim))
    for i in range(num_heads):
        angles = torch.rand(1) * 2 * torch.pi if rotate else torch.zeros(1)        
        fx = torch.cat([mag * torch.cos(angles), mag * torch.cos(torch.pi/2 + angles)], dim=-1)
        fy = torch.cat([mag * torch.sin(angles), mag * torch.sin(torch.pi/2 + angles)], dim=-1)
        freqs_x.append(fx)
        freqs_y.append(fy)
    freqs_x = torch.stack(freqs_x, dim=0)
    freqs_y = torch.stack(freqs_y, dim=0)
    freqs = torch.stack([freqs_x, freqs_y], dim=0)
    return freqs

def reshape_for_broadcast(freqs_cis: torch.Tensor, x: torch.Tensor):
    ndim = x.ndim
    assert 0 <= 1 < ndim
    if freqs_cis.ndim == 2:
        freqs_cis = freqs_cis.unsqueeze(0)
        return freqs_cis
    if freqs_cis.ndim == 3 and freqs_cis.shape[0] == 1: return freqs_cis

    if freqs_cis.shape == (x.shape[-2], x.shape[-1]):
        shape = [d if i >= ndim-2 else 1 for i, d in enumerate(x.shape)]
    elif freqs_cis.shape == (x.shape[-3], x.shape[-2], x.shape[-1]):
        shape = [d if i >= ndim-3 else 1 for i, d in enumerate(x.shape)]
    return freqs_cis.view(*shape)

def apply_rotary_emb(x: torch.Tensor, freqs_cis: torch.Tensor):
    x = x.contiguous()
    x_ = torch.view_as_complex(x.float().reshape(*x.shape[:-1], -1, 2))
    freqs_cis = reshape_for_broadcast(freqs_cis, x_)
    x_out = torch.view_as_real(x_ * freqs_cis).flatten(3)
    return x_out.type_as(x).to(x.device)

from functools import partial

class RoPE(nn.Module):
    def __init__(self, *args, out_channels:int, rope_theta=10.0, in_channels:Optional[int]=None, isproj:bool=False, **kwargs):
        super().__init__(*args, **kwargs)
        assert out_channels % 4 == 0
        self.out_channels = out_channels
        self.num_heads = 1
        self.rope_theta = rope_theta

        self.compute_cis = partial(compute_mixed_cis, num_heads=self.num_heads)     
        freqs = init_2d_freqs(
            dim=self.out_channels // self.num_heads, num_heads=self.num_heads, theta=rope_theta, 
            rotate=True
        ).view(2, -1)
        self.freqs = nn.Parameter(freqs, requires_grad=True) # (2, out_channels // 2)
        
        # t_x, t_y = init_t_xy(end_x=n_x, end_y=n_y) # (n_x * n_y)
        # self.register_buffer('freqs_t_x', t_x)
        # self.register_buffer('freqs_t_y', t_y)

        self.isproj = isproj
        if self.isproj: 
            if in_channels is None: in_channels = out_channels 
            self.proj = nn.Linear(in_channels, out_channels) 

    def forward(self, x:Tensor):
        B, C, *data_shape = x.shape
        N = torch.prod(torch.tensor(data_shape)).item()
        
        x = x.view(B, C, N).permute(0, 2, 1) # (B, N, C)
        if self.isproj: x = self.proj(x)

        ## Rotary position embedding
        
        # if self.freqs_t_x.shape[0] != x.shape[1] - 1:
        if not hasattr(self, 'freqs_t_x') or self.freqs_t_x.shape[0] != x.shape[1]:
            n_x, n_y = data_shape[-2:]
            t_x, t_y = init_t_xy(end_x=n_x, end_y=n_y)
            t_x, t_y = t_x.to(x.device), t_y.to(x.device)
            self.freqs_t_x, self.freqs_t_y = t_x, t_y
        else: 
            t_x, t_y = self.freqs_t_x, self.freqs_t_y
        freqs_cis = self.compute_cis(self.freqs, t_x, t_y)

        x = apply_rotary_emb(x, freqs_cis)

        out = x.permute(0,2,1,3).reshape(B, self.out_channels, *data_shape)
        return out
    
def positionalencoding2d(x:Tensor, d_model:int=None):
    """
    https://github.com/wzlxjtu/PositionalEncoding2D/blob/master/positionalembedding2d.py
    :param d_model: dimension of the model
    :param height: height of the positions
    :param width: width of the positions
    :return: d_model*height*width position matrix
    """
    batch_size, d_model, height, width = x.shape
    n_positional_dims = 2
    if d_model % 4 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                        "odd dimension (got dim={:d})".format(d_model))
    pe = torch.zeros(d_model, height, width)
    # Each dimension use half of d_model

    div_term = torch.exp((torch.arange(0., d_model, 2) / d_model) *
                        -(math.log(10000.0))).to(x.device)
    pos_w = torch.linspace(0, 1, steps=width, device=x.device).unsqueeze(1) # torch.arange(0., width).unsqueeze(1) # 
    pos_h = torch.linspace(0, 1, steps=height, device=x.device).unsqueeze(1)
    pe[0::2, :, :] = torch.cos(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1) - torch.sin(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
    
    pe[1::2, :, :] = torch.sin(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1) - torch.cos(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
    pe = pe.unsqueeze(0).repeat(batch_size, 1, *[1]*n_positional_dims)
    return pe

def _positional_embedding(x):
    '''
    Append spatial positional embedding to the input tensor.
    Parameters:
    - x (torch.tensor): input tensor, shape (batch_size, channels, *spatial_shape)
    
    Returns:
    - x (torch.tensor): output tensor with positional embedding, shape (batch_size, channels + n_positional_dims, *spatial_shape)
    '''
    batch_size = x.shape[0]
    spatial_shape = x.shape[2:]
    n_positional_dims = len(spatial_shape)

    # Create coordinate grids
    grids = torch.meshgrid([torch.linspace(0, 1, steps=s, device=x.device) for s in spatial_shape], indexing='ij')
    # Stack and expand to batch size
    pos_embedding = torch.stack(grids, dim=0).unsqueeze(0).repeat(batch_size, 1, *[1]*n_positional_dims)

    return pos_embedding

class CustomConv2d(nn.Module):
    def __init__(self, 
        in_channels: int, 
        out_channels: int, 
        kernel_size: int = 3, 
        stride: int = 1, 
        padding: int = 1,
        padding_mode: Union[str, Iterable] = 'circular',
        dilation: int = 1,
        bias: bool = True,
        groups: int = 1,
        device: Optional[torch.device] = None,
        dtype: Optional[torch.dtype] = None,
        ):
        super().__init__()
        self.padding = padding
        self.padding_mode = padding_mode
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=padding, padding_mode=padding_mode, dilation=dilation, groups=groups, bias=bias, device=device, dtype=dtype)

    def calc_pad(self, input_shape, output_shape):
        if len(input_shape) > len(output_shape):
            input_shape = input_shape[-len(output_shape):]
        pad = []
        for in_size, out_size in zip(reversed(input_shape), reversed(output_shape)):
            total_pad = max(out_size - in_size, 0)
            pad_left = total_pad // 2 
            pad_right = total_pad - pad_left # If odd, pad more on the right
            pad.extend([pad_left, pad_right])
        return tuple(pad)

    def pad(self, x, pad, mode='constant', value=0):
        dim = len(pad) // 2 

        if mode == 'circular':
            # x 마지막 dim개의 크기만 보고 반복
            input_shape = x.shape[-dim:]
            new_sizes = []
            reps = []
            for i in range(dim):
                left = pad[2*i]
                right = pad[2*i + 1]
                orig = input_shape[-(i+1)]
                new_sz = orig + left + right
                new_sizes.append(new_sz)
                reps.append(np.ceil(new_sz / orig))

            # repeat_sizes는 앞쪽 차원까지 고려
            front_shape_len = x.ndim - dim
            repeat_sizes = [1] * front_shape_len + reps[::-1]
            x_rep = x.repeat(*repeat_sizes)

            # 가운데 영역 slicing
            slices = []
            for i in range(dim):
                orig = input_shape[-(i+1)]
                new_sz = new_sizes[i]
                rep_count = reps[i]
                start = (rep_count * orig - new_sz) // 2
                slices.append(slice(start, start + new_sz))
            slices = slices[::-1]

            out_slices = (Ellipsis,) + tuple(slices)
            return x_rep[out_slices]
        else:
            # circular 아닌 경우
            input_shape = x.shape[-dim:]
            pad_pairs = [(pad[2*i], pad[2*i+1]) for i in range(dim)]
            pad_pairs = pad_pairs[::-1]

            for i, (left, right) in enumerate(pad_pairs):
                cur_left, cur_right = left, right
                current_dim_size = x.shape[-(i+1)]

                if mode == 'reflect':
                    max_once = current_dim_size - 1
                elif mode == 'replicate':
                    max_once = current_dim_size
                else:
                    max_once = current_dim_size

                while cur_left > 0:
                    this_pad = min(cur_left, max_once)
                    pad_config = [0]*(2*dim)
                    pad_config[2*i] = this_pad
                    if mode == 'constant':
                        x = F.pad(x, pad_config, mode=mode, value=value)
                    else:
                        x = F.pad(x, pad_config, mode=mode)
                    cur_left -= this_pad

                while cur_right > 0:
                    this_pad = min(cur_right, max_once)
                    pad_config = [0]*(2*dim)
                    pad_config[2*i + 1] = this_pad
                    if mode == 'constant':
                        x = F.pad(x, pad_config, mode=mode, value=value)
                    else:
                        x = F.pad(x, pad_config, mode=mode)
                    cur_right -= this_pad

            return x
    
    def forward(self, x: torch.Tensor, output_shape:Optional[tuple]=None) -> torch.Tensor:
        
        if output_shape is not None and tuple(x.shape[-len(output_shape):]) != tuple(output_shape): 
            pad = self.calc_pad(x.shape, output_shape)
            if isinstance(self.padding_mode, str):
                mode = self.padding_mode
                x = self.pad(x, pad, mode=mode)
            else:
                for i, p in enumerate(pad):
                    mode = self.padding_mode[i]
                    x = self.pad(x, (p if j in [2*i, 2*i + 1] else 0 for j in range(len(pad))), mode=mode)
        
        x = self.conv(x)
        return x
    
# class SpectralConv(BaseSpectralConv):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.norm = self.fft_norm

#     def forward(
#         self, x: torch.Tensor, output_shape: Optional[Tuple[int]] = None
#     ):
#         batch_size, channels, *data_shape = x.shape
#         fft_shape = list(data_shape) 
#         if not self.complex_data: fft_shape[-1] = fft_shape[-1] // 2 + 1 
#         fft_dims = list(range(-len(data_shape), 0))
#         shift_dims = fft_dims
#         if not self.complex_data: shift_dims = fft_dims[:-1]

#         mode_sizes = output_shape if output_shape is not None else data_shape

#         ## FNO integral kernel computation
#         if self.fno_block_precision == "half": x = x.half()
#         x = self.fft(x, dim=fft_dims)
#         x = self.fftshift(x, dim=shift_dims)
#         if self.fno_block_precision == "mixed": x = x.chalf()

#         x_fno = torch.zeros([batch_size, self.out_channels, *fft_shape],
#                               device=x.device, 
#                               dtype=torch.chalf if self.fno_block_precision in ["half", "mixed"] else torch.cfloat
#                               )
#         slices_w = self.calc_slice_w(self.n_modes, self.max_n_modes, fft_shape, self.separable)
#         weight = self.weight[slices_w]
#         slices_x = self.calc_slices_x(fft_shape, weight.shape, self.separable)
#         x_fno[slices_x] = self._contract(x[slices_x], weight, separable=self.separable)

#         x_fno = self.zero_padding(x_fno, mode_sizes, self.complex_data)
#         x_fno = self.fftshift(x_fno, dim=shift_dims)
#         x = self.ifft(x_fno, dim=fft_dims)

#         if self.bias is not None: x = x + self.bias
#         return x

#     def calc_slice_w(self, n_modes, max_n_modes, fft_size, separable:bool=False):
#         # if current modes are less than max, start indexing modes closer to the center of the weight tensor
#         starts = [(max_modes - min(size, n_mode)) for (size, n_mode, max_modes) in zip(fft_size, n_modes, max_n_modes)]
#         # if contraction is separable, weights have shape (channels, modes_x, ...)
#         # otherwise they have shape (in_channels, out_channels, modes_x, ...)
#         if separable: 
#             slices_w = [slice(None)] # channels
#         else:
#             slices_w =  [slice(None), slice(None)] # in_channels, out_channels
#         if self.complex_data:
#             slices_w += [slice(start//2, -start//2) if start else slice(start, None) for start in starts]
#         else:
#             # The last mode already has redundant half removed in real FFT
#             slices_w += [slice(start//2, -start//2) if start else slice(start, None) for start in starts[:-1]]
#             slices_w += [slice(None, -starts[-1]) if starts[-1] else slice(None)]
#         return slices_w

#     def calc_slices_x(self, fft_size, weight_shape, separable:bool=False):
#         # if separable conv, weight tensor only has one channel dim
#         if separable:
#             weight_start_idx = 1
#         # otherwise drop first two dims (in_channels, out_channels)
#         else:
#             weight_start_idx = 2
        
#         slices_x =  [slice(None), slice(None)] # Batch_size, channels

#         for all_modes, kept_modes in zip(fft_size, list(weight_shape[weight_start_idx:])):
#             # After fft-shift, the 0th frequency is located at n // 2 in each direction
#             # We select n_modes modes around the 0th frequency (kept at index n//2) by grabbing indices
#             # n//2 - n_modes//2  to  n//2 + n_modes//2       if n_modes is even
#             # n//2 - n_modes//2  to  n//2 + n_modes//2 + 1   if n_modes is odd
#             center = all_modes // 2
#             negative_freqs = kept_modes // 2
#             positive_freqs = kept_modes // 2  + kept_modes % 2

#             # this slice represents the desired indices along each dim
#             slices_x += [slice(center - negative_freqs, center + positive_freqs)]
        
#         if weight_shape[-1] < fft_size[-1]:
#             slices_x[-1] = slice(None, weight_shape[-1])
#         else:
#             slices_x[-1] = slice(None)
#         return slices_x

#     def zero_padding(self, Fx: torch.Tensor, mode_size: tuple, complex_data: bool=False):
#         in_shape = list(Fx.shape[-len(mode_size):])
#         mode_size = list(mode_size)
#         if in_shape == mode_size: return Fx
#         if not complex_data: mode_size[-1] = mode_size[-1] // 2 + 1

#         pad_list = []
#         for i, (in_dim, out_dim) in enumerate(zip(reversed(in_shape), reversed(mode_size))):
#             diff = out_dim - in_dim # max(out_dim - in_dim, 0)
#             left = 0 if not complex_data and i == 0 else diff // 2
#             right = diff - left
#             pad_list.extend([left, right])

#         # Apply pad: F.pad expects pads from last to first dimension
#         if any(diff > 0 for diff in (t - i for t, i in zip(mode_size, in_shape))):
#             Fx = F.pad(Fx, pad_list, mode="constant", value=0)
#         return Fx
    
#     def fft(self, x: torch.Tensor, dim=None, norm=None, complex_data: bool=None) -> torch.Tensor:
#         if dim is None: dim = tuple(range(-x.ndim, 0))
#         if norm is None: norm = self.norm
#         if complex_data is None: complex_data = self.complex_data

#         if complex_data:
#             Fx = torch.fft.fftn(x, dim=dim, norm=norm)
#         else:
#             Fx = torch.fft.rfftn(x, dim=dim, norm=norm)
#         return Fx
    
#     def ifft(self, x: torch.Tensor, dim=None, norm=None, complex_data: bool=None) -> torch.Tensor:
#         if dim is None: dim = tuple(range(-x.ndim, 0))
#         if norm is None: norm = self.norm
#         if complex_data is None: complex_data = self.complex_data

#         if complex_data:
#             Fx = torch.fft.ifftn(x, dim=dim, norm=norm)
#         else:
#             Fx = torch.fft.irfftn(x, dim=dim, norm=norm)
#         return Fx
    
#     def fftshift(self, x: torch.Tensor, dim=None) -> torch.Tensor:
#         if dim is None: dim = tuple(range(-x.ndim, 0))
#         if len(dim) > 1: return torch.fft.fftshift(x, dim=dim)
#         else: return x

class SpectralConv(BaseSpectralConv):
    """
    SpectralConv module with CNN skip connection and normalization options.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(
        self, x: torch.Tensor, output_shape: Optional[Tuple[int]] = None
    ):
        """Generic forward pass for the Factorized Spectral Conv

        Parameters
        ----------
        x : torch.Tensor
            input activation of size (batch_size, channels, d1, ..., dN)

        Returns
        -------
        tensorized_spectral_conv(x)
        """
        batchsize, channels, *mode_sizes = x.shape

        fft_size = list(mode_sizes)
        if not self.complex_data:
            fft_size[-1] = fft_size[-1] // 2 + 1  # Redundant last coefficient in real spatial data
        fft_dims = list(range(-self.order, 0))

        
        if self.resolution_scaling_factor is not None and output_shape is None:
            mode_sizes = tuple([round(s * r) for (s, r) in zip(mode_sizes, self.resolution_scaling_factor)])

        if output_shape is not None:
            mode_sizes = output_shape

        if self.fno_block_precision == "half":
            x = x.half()
        def zero_padding(Fx: torch.Tensor, mode_size: tuple, complex_data: bool=False):
            in_shape = list(Fx.shape[-len(mode_size):])
            mode_size = list(mode_size)
            if in_shape == mode_size: return Fx
            if not complex_data: mode_size[-1] = mode_size[-1] // 2 + 1

            pad_list = []
            for i, (in_dim, out_dim) in enumerate(zip(reversed(in_shape), reversed(mode_size))):
                diff = out_dim - in_dim # max(out_dim - in_dim, 0)
                left = 0 if not complex_data and i == 0 else diff // 2
                right = diff - left
                pad_list.extend([left, right])

            # Apply pad: F.pad expects pads from last to first dimension
            if any(diff > 0 for diff in (t - i for t, i in zip(mode_size, in_shape))):
                Fx = F.pad(Fx, pad_list, mode="constant", value=0)
            return Fx


        if self.complex_data:
            x = torch.fft.fftn(x, norm=self.fft_norm, dim=fft_dims)
            dims_to_fft_shift = fft_dims
        else: 
            x = torch.fft.rfftn(x, norm=self.fft_norm, dim=fft_dims)
            # When x is real in spatial domain, the last half of the last dim is redundant.
            # See :ref:`fft_shift_explanation` for discussion of the FFT shift.
            dims_to_fft_shift = fft_dims[:-1] 
        
        
        def calc_slice_w(n_modes, max_n_modes, fft_size, separable:bool=False):
            # if current modes are less than max, start indexing modes closer to the center of the weight tensor
            starts = [(max_modes - min(size, n_mode)) for (size, n_mode, max_modes) in zip(fft_size, n_modes, max_n_modes)]
            # if contraction is separable, weights have shape (channels, modes_x, ...)
            # otherwise they have shape (in_channels, out_channels, modes_x, ...)
            if separable: 
                slices_w = [slice(None)] # channels
            else:
                slices_w =  [slice(None), slice(None)] # in_channels, out_channels
            if self.complex_data:
                slices_w += [slice(start//2, -start//2) if start else slice(start, None) for start in starts]
            else:
                # The last mode already has redundant half removed in real FFT
                slices_w += [slice(start//2, -start//2) if start else slice(start, None) for start in starts[:-1]]
                slices_w += [slice(None, -starts[-1]) if starts[-1] else slice(None)]
            return slices_w

        def calc_slices_x(fft_size, weight_shape, separable:bool=False):
            # if separable conv, weight tensor only has one channel dim
            if separable:
                weight_start_idx = 1
            # otherwise drop first two dims (in_channels, out_channels)
            else:
                weight_start_idx = 2
            
            slices_x =  [slice(None), slice(None)] # Batch_size, channels

            for all_modes, kept_modes in zip(fft_size, list(weight_shape[weight_start_idx:])):
                # After fft-shift, the 0th frequency is located at n // 2 in each direction
                # We select n_modes modes around the 0th frequency (kept at index n//2) by grabbing indices
                # n//2 - n_modes//2  to  n//2 + n_modes//2       if n_modes is even
                # n//2 - n_modes//2  to  n//2 + n_modes//2 + 1   if n_modes is odd
                center = all_modes // 2
                negative_freqs = kept_modes // 2
                positive_freqs = kept_modes // 2  + kept_modes % 2

                # this slice represents the desired indices along each dim
                slices_x += [slice(center - negative_freqs, center + positive_freqs)]
            
            if weight_shape[-1] < fft_size[-1]:
                slices_x[-1] = slice(None, weight_shape[-1])
            else:
                slices_x[-1] = slice(None)
            return slices_x
        ### fno layer
        if self.order > 1:
            x = torch.fft.fftshift(x, dim=dims_to_fft_shift)

        if self.fno_block_precision == "mixed":
            # if 'mixed', the above fft runs in full precision, but the
            # following operations run at half precision
            x = x.chalf()
            

        if self.fno_block_precision in ["half", "mixed"]:
            out_dtype = torch.chalf
        else:
            out_dtype = torch.cfloat
        out_fft = torch.zeros([batchsize, self.out_channels, *fft_size],
                              device=x.device, dtype=out_dtype)
        
        slices_w = calc_slice_w(self.n_modes, self.max_n_modes, fft_size, self.separable)
        weight = self.weight[slices_w]
        slices_x = calc_slices_x(fft_size, weight.shape, self.separable)
        out_fft[slices_x] = self._contract(x[slices_x], weight, separable=self.separable)

        out_fft = zero_padding(out_fft, mode_sizes, self.complex_data)

        
        if self.order > 1:
            out_fft = torch.fft.fftshift(out_fft, dim=dims_to_fft_shift)
        if self.complex_data:
            x = torch.fft.ifftn(out_fft, dim=fft_dims, norm=self.fft_norm) # x = torch.fft.ifftn(x_fft, dim=fft_dims, norm=self.fft_norm).real
        else:
            x = torch.fft.irfftn(out_fft, dim=fft_dims, norm=self.fft_norm)
        if self.bias is not None:
            x = x + self.bias

        return x


class FNOBlocks(BaseFNOBlocks):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.frozen_layer = []

        self.cnn_hidden_channels = int(self.out_channels * 1)
        self.cnn_n_layer = 2
        cnn_skip = nn.ModuleList()
        cnn1x1 = nn.ModuleList()
        for _ in range(self.n_layers):
            cnn_layer = nn.ModuleList()
            for i in range(self.cnn_n_layer):
                cnn_layer.append(WTConv2d(self.in_channels if i == 0 else self.cnn_hidden_channels, 
                                          self.cnn_hidden_channels, kernel_size=3, stride=1, bias=False, wt_levels=2, wt_type='db2'))
                # cnn_layer.append(CustomConv2d(self.in_channels if i == 0 else self.cnn_hidden_channels, 
                #                               self.cnn_hidden_channels, kernel_size=3, stride=1, padding=1, padding_mode='circular'))
                cnn_layer.append(nn.InstanceNorm2d(self.cnn_hidden_channels))
                cnn_layer.append(nn.GELU())
            cnn_layer = nn.Sequential(*cnn_layer)
            cnn_skip.append(cnn_layer)

            cnn1x1.append(nn.Sequential(nn.Conv2d(
                self.out_channels + self.cnn_hidden_channels, # + self.in_channels,
                self.out_channels, kernel_size=1, 
            )))
        self.cnn_skip = nn.ModuleList(cnn_skip)
        self.cnn1x1 = nn.ModuleList(cnn1x1)
    
    def forward(self, x, index=0, output_shape=None, positional_encoder=None):
        if positional_encoder is not None: x = positional_encoder(x)
        if not 'fno' in self.frozen_layer: 
            x_fno = self.convs[index](x, output_shape=output_shape)
            if self.norm is not None: x_fno = self.norm[self.n_norms * index](x_fno)
            # x_fno += x
            if hook: hook(x_fno, 'x_fno')
        if not 'cnn' in self.frozen_layer: 
              
            # x = self.low_pass_filter(x)
            if hook: hook(x, 'hpf(x)')
            x_cnn = self.cnn_skip[index](x)
            # if hook: hook(x_cnn, 'cnn(hpf(x))')
            # x_cnn = self.low_pass_filter(x_cnn)
            # if hook: hook(x_cnn, 'hpf(cnn(hpf(x)))')
            
            if hook: hook(x_cnn, 'x_cnn')
            # x = x_fno + x_cnn
            # x_cnn = x * x_cnn
            x = torch.concat([x_fno, x_cnn], dim=1) # x = x_fno
            # x = torch.concat([x_fno, x_fno * x_cnn, x_cnn, x], dim=1)
            # x = x_fno + x * x_cnn
            if hook: hook(x, 'x_fno + x_cnn')
            
            x = self.non_linearity(x)
            if hook: hook(x, 'non_linearity')
            x = self.cnn1x1[index](x)
            if hook: hook(x, 'act')
            # x += x_fno
        else: 
            x = x_fno
        if positional_encoder is not None: x = positional_encoder(x)
        return x
    
    def low_pass_filter(self, x):
            def calc_slices_x(fft_size, n_modes, max_n_modes, complex_data=False):
                """
                Return a list of slice objects for indexing the input x (after FFT) 
                to select only the required modes in each axis.

                Parameters:
                    fft_size (tuple): FFT size along each dimension.
                    n_modes (tuple): Number of modes to keep along each dimension.
                    max_n_modes (tuple): Maximum available modes along each dimension.
                    complex_data (bool): Whether input is complex (True) or real (False).

                Returns:
                    slices_x (list of slice): Slices for indexing x to select wanted modes.
                """
                # Compute starting indices for each dimension
                starts = [(max_modes - min(size, n_mode)) for (size, n_mode, max_modes) in zip(fft_size, n_modes, max_n_modes)]

                # Always slice batch and channel as full
                slices_x = [slice(None), slice(None)]

                # For each dimension, select the center region (kept modes)
                for dim, (size, n_mode, start) in enumerate(zip(fft_size, n_modes, starts)):
                    # Center index after fftshift
                    center = size // 2
                    kept_modes = min(size, n_mode)
                    neg_freqs = kept_modes // 2
                    pos_freqs = kept_modes // 2 + kept_modes % 2

                    # Final slice for this dimension
                    slices_x.append(slice(center - neg_freqs, center + pos_freqs))

                # If real input, last FFT axis is special (half spectrum is redundant)
                if not complex_data:
                    # For real FFT, only positive frequencies along the last axis are kept
                    last_dim = len(fft_size) + 1  # +2 for batch, channel; -1 for zero-indexing
                    last_mode = n_modes[-1]
                    slices_x[-1] = slice(0, last_mode)

                return slices_x 
            
            Fx = torch.fft.fftn(x, dim=(-2, -1), norm='forward')
            Fx = torch.fft.fftshift(Fx, dim=(-2, -1))
            idx = calc_slices_x(list(Fx.shape[2:]), n_modes=self.n_modes, max_n_modes=self.n_modes, complex_data=True)
            Fx[idx] = 0. + 0j
            Fx = torch.fft.fftshift(Fx, dim=(-2, -1))
            x = torch.fft.ifftn(Fx, dim=(-2, -1), norm='forward').real
            return x
    
class FNO(BaseFNO):
    def __init__(self, 
        non_linearity: callable = nn.GELU(), # CNO_activation(activation=F.gelu), Dealiasing_activation(activation=F.gelu), #         
        *args, **kwargs):
        super().__init__(conv_module=SpectralConv, non_linearity=non_linearity, *args, **kwargs)
        kwargs.pop('in_channels', None)
        kwargs.pop('out_channels', None)
        self.fno_blocks = FNOBlocks(
            in_channels=self.hidden_channels, 
            out_channels=self.hidden_channels,
            conv_module=SpectralConv, 
            non_linearity=non_linearity,
            *args, **kwargs
            )
        
        
        self.projection = ChannelMLP(
            in_channels=self.hidden_channels,
            out_channels=self.out_channels,
            hidden_channels=self.projection_channels,
            n_layers=4,
            n_dim=self.n_dim,
            non_linearity=non_linearity,
        )
        if self.complex_data:
            self.projection = ComplexValued(self.projection)

        self.positional_encoder = RoPE(out_channels=self.hidden_channels, rope_theta=10.0, isproj=False, )
        self.frozen_layer = []
    
    # def forward(self, x_branch: torch.Tensor, x_trunk: torch.Tensor, *args, **kwargs):
    def forward(self, x, output_shape=None, *args, **kwargs):
        # self.frozen_layer = ['fno']
        if output_shape is None:
            output_shape = [None]*self.n_layers
        elif isinstance(output_shape, tuple):
            output_shape = [None]*(self.n_layers - 1) + [output_shape]

        if hook: hook(x, 'input')
        if not 'ae' in self.frozen_layer: 
            x = self.lifting(x)     # (batch_size, output_dim)
            if hook: hook(x, 'lifting')
        if False: 
            x = self.positional_encoder(x)
            # x *= pe.to(x.device)
            if hook: hook(x, 'x + pe')

        if not 'fno' in self.frozen_layer: 
            for layer_idx in range(self.n_layers):
                if hook: hook.set_prefix(f'FNO Layer {layer_idx}')
                x = self.fno_blocks(x, layer_idx, output_shape=output_shape[layer_idx], 
                                    # positional_encoder=self.positional_encoder
                                    )

                # x *= pe.to(x.device)
            if hook: hook.set_prefix('')
            
        # Element-wise multiplication and summation over the output dimension
        if False:
            pos_embedding = self._positional_embedding(x)
            x_branch = self.lifting_branch(pos_embedding)  # (batch_size, output_dim)
            x_branch *= pe.to(x_branch.device)
            x = x_branch * x  # (batch_size, 1  )   
            if hook: hook(x, 'x + x_branch')
        if not 'ae' in self.frozen_layer: 
            x = self.projection(x)
            if hook: hook(x, 'projection')
        return x
    
    def freeze_layers(self, layer:Literal['AE', 'fno', 'cnn', 'all']):
        """
        Freeze specific layers of the model.

        Parameters
        ----------
        layer : Literal['AE', 'fno', 'cnn', 'all']
            The layer(s) to freeze.
        """
        if layer == 'AE':
            for param in self.lifting.parameters():
                param.requires_grad = False
            for param in self.projection.parameters():
                param.requires_grad = False
        elif layer == 'fno':
            for param in self.fno_blocks.parameters():
                param.requires_grad = False
        elif layer == 'cnn':
            for block in self.fno_blocks.convs:
                if isinstance(block.cnn_skip, ScaledCNNLayer):
                    for param in block.cnn_skip.parameters():
                        param.requires_grad = False
                    block.cnn1x1.requires_grad = False
                    
        elif layer == 'all':
            for param in self.parameters():
                param.requires_grad = False
        else:
            raise ValueError(f"Unknown layer type: {layer}")
    
    def unfreeze_layers(self):
        """
        Unfreeze all layers of the model.
        """
        for param in self.parameters():
            param.requires_grad = True

In [8]:
import torch
import torch_optimizer 
from utils.refer.soap import SOAP

SCHEDULERS = {
    'StepLR': lambda optimizer, **params: torch.optim.lr_scheduler.StepLR(
        optimizer, **(params | {'step_size': 10, 'gamma': 0.1})
    ),
    'ExponentialLR': lambda optimizer, **params: torch.optim.lr_scheduler.ExponentialLR(
        optimizer, **(params | {'gamma': 0.95})
    ),
    'MultiStepLR': lambda optimizer, **params: torch.optim.lr_scheduler.MultiStepLR(
        optimizer, **(params | {'milestones': [30, 80], 'gamma': 0.1})
    ),
    'CosineAnnealingLR': lambda optimizer, **params: torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, **(params | {'T_max': 10})
    ),
    'CosineAnnealingWarmRestarts': lambda optimizer, **params: torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, **(params | {'T_0': 10, 'T_mult': 2})
    ),
    'OneCycleLR': lambda optimizer, **params: torch.optim.lr_scheduler.OneCycleLR(
        optimizer, **(params | {'max_lr': 0.1, 'total_steps': 100})
    ),
    'ReduceLROnPlateau': lambda optimizer, **params: torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, **(params | {'mode': 'min', 'factor': 0.1, 'patience': 10})
    ),
}


OPTIMIZERS = {
    'SGD': lambda model, **params: torch.optim.SGD(
        model, **({'lr': 0.01, 'momentum': 0.9} | params)
    ),
    'Adam': lambda model, **params: torch.optim.Adam(
        model, **({'lr': 1e-3, 'betas': (0.9, 0.999)} | params)
    ),
    'AdamW': lambda model, **params: torch.optim.AdamW(
        model, **({'lr': 1e-3, 'weight_decay': 1e-2} | params)
    ),
    'RMSprop': lambda model, **params: torch.optim.RMSprop(
        model, **({'lr': 1e-2, 'alpha': 0.99, 'momentum': 0.9} | params)
    ),
    'Adagrad': lambda model, **params: torch.optim.Adagrad(
        model, **({'lr': 1e-2} | params)
    ),
    'Adadelta': lambda model, **params: torch.optim.Adadelta(
        model, **({'lr': 1.0, 'rho': 0.9} | params)
    ),
    'Adamax': lambda model, **params: torch.optim.Adamax(
        model, **({'lr': 2e-3} | params)
    ),
    'NAdam': lambda model, **params: torch.optim.NAdam(
        model, **({'lr': 2e-3, 'betas': (0.9, 0.999)} | params)
    ),
    'Shampoo': (
        (lambda model, **params: torch_optimizer.Shampoo(
            model, **({'lr': 1e-3, 'momentum': 0.9, 'weight_decay': 0.0} | params)
        )) 
    ),
    'SOAP': (
        (lambda model, **params: SOAP(
            model, **({'lr': 1e-3,} | params)
        )) 
    ),
}


        

class CustomModule(nn.Module):
    def __init__(self, 
                 model:Optional[callable]= None,
                 modelconfig:Optional[dict]=None, 
                 optconfig:Optional[dict]=None, 
                 
                 loss_fn: Optional[callable] = nn.MSELoss(),
                 transformer:Optional[callable]=None, 
                 **kwargs, 
                 ):
        super().__init__()
        
        self.modelconfig = modelconfig
        self.optconfig = optconfig
        self.hparams = kwargs
        self.device = device

        self.model = self.build_model() if model is None else model.to(device)

        self.optimizer = None
        self.scheduler = None
        if optconfig is not None: self.configure_optimizers()

        self.loss_fn = loss_fn
        self.transformer = transformer


        self.epoch = 0

    def build_model(self):
        self.model = FNO(**self.modelconfig['params'],).to(self.device)

        return self.model
    
    def configure_optimizers(self):
        if self.optimizer is not None: 
            return self.optimizer, self.scheduler
        self.optimizer = OPTIMIZERS[self.optconfig['name']](
            self.model.parameters(), 
            **self.optconfig['params']
        )

        self.scheduler = None
        if self.optconfig['scheduler']: 
            self.scheduler = SCHEDULERS[self.optconfig['scheduler']['name']](
                self.optimizer, 
                **self.optconfig['scheduler'].get('params', {}),
                ) 
        return self.optimizer, self.scheduler
    
    def forward(self, data, output_shape=None):
        return self.model(data.to(self.device), output_shape=output_shape)

    def training_step(self, batch, batch_idx, ret_log:bool=False):
        if self.transformer: batch = self.transformer(batch)
        data, target = batch
        
        # data = data[:, 0]
        # target = target[:, 0]
        if hook and batch_idx % hook.hook_every_n_iter == 0: hook.open_hook()
        self.optimizer.zero_grad()
        output = self.forward(data.to(self.device), output_shape=target.shape[2:])
        loss = self.loss_fn(output, target.to(self.device))
        loss.backward()
        if hook and batch_idx % hook.hook_every_n_iter == 0: 
            hook.close_hook(fname='./result/' + f'Training iter{batch_idx}')
            hook.step()

        self.optimizer.step()
        
        if batch_idx % 10 == 0: self.epoch += 1
        self.log('loss', loss.item(), on_step=True)
        return loss
    
    @torch.no_grad()
    def validation_step(self, batch, batch_idx, ret_log:bool=True):
        if self.transformer: batch = self.transformer(batch)
        data, target = batch

        # data = data[:, 0]
        # target = target[:, 0]
        if hook and batch_idx == 0: hook.open_hook()
        output = self.forward(data.to(self.device), output_shape=target.shape[2:])
        loss = self.loss_fn(output, target.to(self.device))
        if hook and batch_idx == 0: hook.close_hook(fname='./result/' + f'Valid iter{hook.iteration}_n{target.shape[-1]}-n{target.shape[-1]}')

        self.log('val_loss', loss.item(), on_step=False)
        if ret_log: 
            log = {}
            log.update({"loss": loss.item(), "output": output[:].detach().cpu(), 'batch': [data, target.detach().cpu()]})
            return log
        return output

    @torch.no_grad()
    def test_step(self, batch, batch_idx, ret_log:bool=True):
        if self.transformer: batch = self.transformer(batch)
        
        data, target = batch

        # data = data[:, 0]
        # target = target[:, 0]

        output = self.forward(data.to(self.device), output_shape=target.shape[2:])

        if ret_log: 
            log = {}
            log.update({# "loss": loss.item(), 
                        "output": output[:].detach().cpu(), 'batch': [data.detach().cpu(), target.detach().cpu()]})
            return log
        return output

    def save(self, path, checkpoint:dict={}):
        checkpoint['model_state_dict'] = self.model.state_dict()
        checkpoint['optimizer_state_dict'] = self.optimizer.state_dict()
        if self.scheduler: checkpoint[f'scheduler'] = self.scheduler.state_dict()

        torch.save(checkpoint, path + '.pth')

    def load(self,
        path, 
        verbose=True, 
        load_opt:bool=True,
        freeze:bool=False,
        ):
        if os.path.exists(path + '.pth') and os.path.getsize(path + '.pth'):
            checkpoint = torch.load(path + '.pth', weights_only=False)

            self.model.load_state_dict(
                checkpoint['model_state_dict'], 
                strict=False,
                )
            if freeze: 
                for name, param in self.model.named_parameters():
                    if name in checkpoint['model_state_dict']:
                        param.requires_grad = False
            if verbose: print(f'model loaded from {path}')
            if load_opt:
                if self.optimizer and 'optimizer_state_dict' in checkpoint: 
                    self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                    if verbose: print(f'optimizer loaded from {path}')
                if self.scheduler and 'scheduler' in checkpoint: 
                    self.scheduler.load_state_dict(checkpoint['scheduler'])
                    if verbose: print(f'scheduler loaded from {path}')
    
    def log(self, name, value, prog_bar=False, logger=None, on_step=None, on_epoch=None, reduce_fx='mean', enable_graph=False, sync_dist=False, sync_dist_group=None, add_dataloader_idx=True, batch_size=None, metric_attribute=None, rank_zero_only=False):
        commit = False if on_step is False else True
        wandb.log({name: value}, commit=commit)

In [9]:
from utils.BaseCallback import BaseCallback
from utils.CFDFunction import calc_energy_spectrum, calc_pdf, calc_phase_error
from utils.Plots import plot_2d_surface, plot_spectrum, plot_pdf

class CustomCallback(BaseCallback):
    def __init__(self, 
                 criterions, 
                 file_dir:str='', 
                 fname:str ='',
                 device:str =None, 
                 ):
        super(CustomCallback, self).__init__()
        self.criterions = criterions # DL criterion + CFD criterion + DL part-of-loss
        self.file_dir = file_dir
        self.fname = fname
        self.device=device
        
    def on_validation_epoch_start(self, trainer, pl_module):
        self.time_stamp = time.time()
        trainer._current_val_return = {'loss': [], 'time': []}
        for key in self.criterions.keys():
            trainer._current_val_return[key] = []

    def on_validation_batch_end(self, trainer, pl_module, outputs, batch: Any, batch_idx: int, dataloader_idx: int = 0) -> None:
        data, target = outputs['batch'] # batch
        # trainer._current_val_return['batch'] = outputs['batch'] # batch
        # trainer._current_val_return['output'] = outputs['output']
        # trainer._current_val_return['loss'].append(outputs['loss'])

        for key in self.criterions.keys():
            value = self.criterions[key](outputs['output'].to(self.device), target.to(self.device)).item()

            trainer._current_val_return[key].append(value)
            
        self.N = data.shape[-1]
        if data.shape[-1] != target.shape[-1]: self.N = f'{data.shape[-1]}-{target.shape[-1]}'
    
    def on_validation_end(self, trainer, pl_module, fname='') -> None:
        fname = self.fname + fname
        # data, target = trainer._current_val_return['batch']
        # output = trainer._current_val_return['output']
        # if trainer.logger: trainer.logger.log({'val_loss': np.mean(trainer._current_val_return['loss'])})

        epoch = trainer.current_epoch
        print(f'  Validation Epoch: {epoch}', end='')
        for key in self.criterions.keys():
            value = np.mean(trainer._current_val_return[key])
            print(f", {key}: {value:.6f}", end='')

            if trainer.logger: trainer.logger.log({f'Val_{self.N}/epoch/'+key: value})
        print()

        plt.close('all')

     


    def on_test_batch_start(self, trainer, pl_module, batch: Any, batch_idx: int, dataloader_idx: int = 0) -> None:
        self.time_stamp = time.time()

    def on_test_batch_end(self, trainer, pl_module, outputs, batch: Any, batch_idx: int, dataloader_idx: int = 0) -> None:
        data, target = batch
        trainer._current_test_return['batch'] = batch
        output = trainer._current_test_return['output'] = outputs['output']
        
        runtime = time.time() - self.time_stamp
        if trainer.logger: trainer.logger.log({'Test/time': runtime})
        print(f'Test sample {batch_idx}(time: {runtime:.6f}, batch size: {len(output)})', end='')
        for key in self.criterions.keys():
            value = self.criterions[key](outputs['output'].to(self.device), target.to(self.device)).item()

            trainer._current_test_return[key].append(value)
            print(f", {key}: {value:.6f}", end='')
        print()
        
        ## enstrophy spectrum
        spectrum1 = calc_energy_spectrum(target[:,0,:], dim=(-2, -1), channel_dim=1) # np.mean([calc_enstrophy_spectrum(target[i,0,:]) for i in range(len(target))], axis=0); 
        spectrum2 = calc_energy_spectrum(output[:,0,:], dim=(-2, -1), channel_dim=1) # np.mean([calc_enstrophy_spectrum(output[i,0,:]) for i in range(len(output))], axis=0)
        ## vorticity pdf
        bin1, pdf1 = calc_pdf(target[:,0,:], dim=2) # np.mean([calc_pdf(target[i,0,:]) for i in range(len(target))], axis=0); 
        bin2, pdf2 = calc_pdf(output[:,0,:], dim=2) # np.mean([calc_pdf(output[i,0,:]) for i in range(len(output))], axis=0)

        # ### solution field 
        plot_2d_surface(target[0,0,...,0], output[0,0,...,0], # axes=[ax1,ax2], 
                        kwargs={'figsize': (8, 4)}
                        )
        plt.savefig(self.file_dir + self.fname + f'_Test_sample={batch_idx}_field.png')
        # plt.show()
        plt.close()

        fig = plt.figure(figsize=(8,4))
        ax = fig.add_subplot(1, 2, 1)
        plot_spectrum(spectrum1, spectrum2, ax=ax, kwargs={'title': f'enstrophy spectrum'})

        ax = fig.add_subplot(1, 2, 2)
        plot_pdf(bin1, pdf1, bin2, pdf2, ax=ax, kwargs={'title': f'pdf'})
        plt.tight_layout()
        plt.savefig(self.file_dir + self.fname + f'_Test_sample={batch_idx}_stat.png')
        # plt.show()
        plt.close()

        ## save .wandb
        Nt = len(output)
        for it in range(Nt):
            image = np.concatenate((target[it,0,:], output[it,0,:]), axis=1)
            if trainer.logger: trainer.logger.log({'Test/'+f'sample_{batch_idx}/'+'field': wandb.Image(image)})
        if trainer.logger: trainer.logger.log({'Test/spectrum': wandb.plot.line_series(
            xs=[np.arange(0, spectrum1.shape[-1]+1), np.arange(0, spectrum2.shape[-1]+1)], 
            ys = [spectrum1, spectrum2],
            keys = ['ground truth', 'prediction'],
            xname = ['k', 'enstrophy spectrum'],
            title='enstrophy_spectrum', 
            )})
        if trainer.logger: trainer.logger.log({'Test/pdf': wandb.plot.line_series(
            xs=[bin1, bin2], 
            ys = [pdf1, pdf2],
            keys = ['ground truth', 'prediction'],
            xname = ['w', 'pdf'],
            title='pdf', 
            )})

    def on_test_end(self, trainer, pl_module) -> None:
        data, target = trainer._current_test_return['batch']
        output = trainer._current_test_return['output']

        print('Test ')
        for key in self.criterions.keys():
            value = np.mean(trainer._current_test_return[key])
            print(f", {key}: {value:.6f}", end='')

            if trainer.logger: trainer.logger.log({'Test/'+key: value})
        print()

    def preprocessing(self, x):
        n_modes = self.n_modes
        x = self.split(x, modes=n_modes)
        # x = self.scaling(x, dim=2, isNormalize=False)
        return x 
    
    def postprocessing(self, x):
        n_modes = self.n_modes
        # x = self.unscaling(x)
        x = self.unsplit(x, modes=n_modes)
        return x
    
    def scaling(self, x, 
                dim=1, 
                isNormalize:bool=False):
        dim = list(range(-dim, 0))
        if isNormalize:
            self.mean = x.amin(dim=dim, keepdim=True)
            self.std = x.amax(dim=dim, keepdim=True) - self.mean
        else: 
            self.mean = x.mean(dim=dim, keepdim=True)
            self.std = x.std(dim=dim, keepdim=True)
        return (x - self.mean) / self.std
    
    def unscaling(self, x):
        return x * self.std + self.mean
    
    def split(self, x, 
              modes=None
              ):

        b, c, *data_shape = x.shape
        fft_dims = list(range(-len(data_shape), 0))
        Fx = torch.fft.fft2(x, dim=fft_dims, norm='forward')
        Fx_ = torch.zeros((len(modes), b, c, *data_shape), dtype=Fx.dtype, device=x.device)
        
        k = [torch.fft.fftfreq(n, d=1./n) for n in data_shape]
        # k += [torch.fft.rfftfreq(data_shape[-1], d=1./data_shape[-1])] if Fx.dtype in [torch.float16, torch.float32, torch.float64] else [torch.fft.fftfreq(data_shape[-1], d=1./data_shape[-1])]


        k = torch.meshgrid(k, indexing='ij')
        k = torch.stack(k)
        k = torch.sqrt(torch.sum(k**2, axis=0)).to(x.device)
        for i in range(len(modes)):
            k1, k2 = modes[i]
            idx = (k1 <= k) & (k <= k2)
            Fx_[i] = Fx * idx

        x_ = torch.fft.ifft2(Fx_, dim=fft_dims, norm='forward').real # x_ = Fx_ # # 
        
        return x_

    def unsplit(self, x_, 
              modes=None
              ):
        n, b, c, *data_shape = x_.shape
        fft_dims = list(range(-len(data_shape), 0))
        
        Fx_ = torch.fft.fft2(x_, dim=fft_dims, norm='forward') # Fx_ = x_ # 
        
        Fx = torch.zeros((b, c, *data_shape), dtype=Fx_.dtype, device=x_.device)
        k = [torch.fft.fftfreq(n, d=1./n) for n in data_shape[:-1]]
        k += [torch.fft.rfftfreq(data_shape[-1], d=1./data_shape[-1])] if Fx.dtype in [torch.float16, torch.float32, torch.float64] else [torch.fft.fftfreq(data_shape[-1], d=1./data_shape[-1])]
        k = torch.meshgrid(k, indexing='ij')
        k = torch.stack(k)
        k = torch.sqrt(torch.sum(k**2, axis=0)).to(x_.device)
        for i in range(len(modes)):
            k1, k2 = modes[i]
            idx = (k1 <= k) & (k <= k2)
            Fx[..., idx] = Fx_[i, ..., idx]

        x = torch.fft.ifft2(Fx, dim=fft_dims, norm='forward').real
        return x

class FineTunner_1(BaseCallback):
    def __init__(self, loss_fn):
        super().__init__()
        self.loss_fn = loss_fn

        self.phase = 1
        self.eps = [5e-6, 1e-5]
        
        self.loss_prev = None

    def on_fit_start(self, trainer, pl_module):
        pl_module.model.frozen_layer.append('fno')
        trainer.train_data_idx = [0]

    def on_validation_end(self, trainer, pl_module):
        if self.phase == 2 and 'fno' in pl_module.model.frozen_layer:
            pl_module.model.frozen_layer.remove('fno')
            for param in pl_module.model.lifting.parameters():
                param.requires_grad = False
            for param in pl_module.model.projection.parameters():
                param.requires_grad = False
            
            pl_module.model.fno_blocks.frozen_layer.append('cnn')
            trainer.train_data_idx = [1]
            trainer.val_data_idx.remove(0)
            trainer.val_data_idx.remove(1)
            
        if self.phase == 3 and 'cnn' in pl_module.model.fno_blocks.frozen_layer:
            pl_module.model.fno_blocks.frozen_layer.remove('cnn')
            # for param in pl_module.model.fno_blocks.convs.parameters():
            #     param.requires_grad = False
            # for param in pl_module.model.fno_blocks.norm.parameters(): 
            #     param.requires_grad = False

    # def forward(self, pl_module.model, loss):
    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
        loss = outputs
        if self.loss_prev is None: 
            self.loss_prev = loss 
            return 
        
        res = torch.abs(loss - self.loss_prev).item() 
        if self.phase == 1 and res <= self.eps[0]:
            pl_module.save(path='./checkpoint/FineTunner_Phase1')
            print(f'Fine tunning phase {self.phase} Ended. res:', res, 'epoch:', trainer.current_epoch)
            self.phase += 1
            self.loss_prev = None   
            trainer.skip_current_epoch = True
        elif self.phase == 2 and res <= self.eps[1]:
            pl_module.save(path='./checkpoint/FineTunner_Phase2')
            print(f'Fine tunning phase {self.phase} Ended. res:', res, 'epoch:', trainer.current_epoch)
            self.phase += 1
            self.loss_prev = None
            trainer.skip_current_epoch = True
        self.loss_prev = loss
        wandb.log({'fine_tunner_res': res})

class FineTunner(BaseCallback):
    def __init__(self, loss_fn):
        super().__init__()
        self.loss_fn = loss_fn

        self.phase = 1
        self.eps = [0.3, 1e-3]
        
        self.loss_prev = None

    def on_fit_start(self, trainer, pl_module):
        pl_module.model.fno_blocks.frozen_layer.append('cnn')
        # trainer.train_data_idx = [1]

    def on_validation_end(self, trainer, pl_module):
            
        if self.phase == 2 and 'cnn' in pl_module.model.fno_blocks.frozen_layer:
            pl_module.model.fno_blocks.frozen_layer.remove('cnn')
            for param in pl_module.model.fno_blocks.convs.parameters():
                param.requires_grad = False
            for param in pl_module.model.fno_blocks.norm.parameters(): 
                param.requires_grad = False
            for param in pl_module.model.lifting.parameters():
                param.requires_grad = False
            for param in pl_module.model.projection.parameters():
                param.requires_grad = False

    # def forward(self, pl_module.model, loss):
    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
        loss = outputs
        if self.loss_prev is None: 
            self.loss_prev = loss 
            return 
        
        res = loss # torch.abs(loss - self.loss_prev).item() 
        if self.phase == 1 and res <= self.eps[0]:
            pl_module.save(path='./checkpoint/FineTunner_Phase1')
            print(f'Fine tunning phase {self.phase} Ended. res:', res, 'epoch:', trainer.current_epoch)
            self.phase += 1
            self.loss_prev = None   
            trainer.skip_current_epoch = True
        self.loss_prev = loss
        wandb.log({'fine_tunner_res': res})

In [10]:
##### set parameters #####
### save parameters ### 
data_dir = config['data'][0]['params']['base_path'] # '../Data/nu=0.001_n=128/'
data_fname = config['data'][0]['params']['dataset_name'] # f'2dHIT_nu=0.001_n=128_T=11.5'

metadata = HIT2dDataset(path=data_dir + 'train/' + data_fname, load_data=False).metadata
print(metadata)

nu = metadata['nu']
dt = metadata['dt']
N = metadata['Nx']

Loaded dataset metadata: dict_keys(['nu', 'Lx', 'Ly', 'dx', 'dy', 'Nx', 'Ny', 'x', 'y', 'dt', 'Nt', 'T', 't0', 't'])
{'nu': 0.000225, 'Lx': 1.5402125848364265, 'Ly': 1.5402125848364265, 'dx': 0.024447818806927406, 'dy': 0.024447818806927406, 'Nx': 64, 'Ny': 64, 'x': array([0.        , 0.02444782, 0.04889564, 0.07334346, 0.09779128,
       0.12223909, 0.14668691, 0.17113473, 0.19558255, 0.22003037,
       0.24447819, 0.26892601, 0.29337383, 0.31782164, 0.34226946,
       0.36671728, 0.3911651 , 0.41561292, 0.44006074, 0.46450856,
       0.48895638, 0.51340419, 0.53785201, 0.56229983, 0.58674765,
       0.61119547, 0.63564329, 0.66009111, 0.68453893, 0.70898675,
       0.73343456, 0.75788238, 0.7823302 , 0.80677802, 0.83122584,
       0.85567366, 0.88012148, 0.9045693 , 0.92901711, 0.95346493,
       0.97791275, 1.00236057, 1.02680839, 1.05125621, 1.07570403,
       1.10015185, 1.12459967, 1.14904748, 1.1734953 , 1.19794312,
       1.22239094, 1.24683876, 1.27128658, 1.2957344 , 1.320182

In [11]:
from utils.Losses import (RMSLoss, TKELoss, DissipationLoss, RelambdaLoss, R2, 
                          BSMSE, )
from utils.utilities import HsLoss
from utils.Losses import BaseLoss

class FourierCorrLoss(BaseLoss):
    '''
    L2 based cos similarity in Fourier space
    '''
    def __init__(self, 
        isRelative:bool=False, 
        norm:Literal['forward', 'ortho', 'backward']='forward', 
        dim:Union[int, Iterable]=(-2, -1), 
        *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.isRelative = isRelative
        self.dim = dim if isinstance(dim, Iterable) else (dim,)
        self.norm = norm
    
    def forward(self, input:Tensor, target:Tensor) -> Tensor:
        F_in = torch.fft.fftn(input, norm=self.norm, dim=self.dim)
        F_tar = torch.fft.fftn(target, norm=self.norm, dim=self.dim)

        dim = [-(len(self.dim)+1)] + list(self.dim)
        Sxy = (F_in * torch.conj(F_tar)).sum(dim=dim)
        loss = torch.abs(Sxy)
        if self.isRelative: 
            # norm = torch.sqrt(torch.abs(F_tar).mean(dim=dim)) * torch.sqrt(torch.abs(F_in).mean(dim=dim))
            norm = torch.linalg.vector_norm(F_in, dim=dim) * torch.linalg.vector_norm(F_tar, dim=dim)
            loss = loss / (norm + 1e-8)
            loss = - torch.log(loss) # loss = 1.0 - loss

        if self.reduction == 'mean': loss = loss.mean()
        elif self.reduction =='sum': loss = loss.sum()

        return loss

class FourierWeightedMSELoss(BaseLoss):
    '''
    L2 based cos similarity in Fourier space
    '''
    def __init__(self, 
        # norm:Literal['forward', 'ortho', 'backward']='ortho', 
        dim:Union[int, Iterable]=(-2, -1), 
        *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.dim = dim if isinstance(dim, Iterable) else (dim,)
        self.norm = 'ortho'
    
    def forward(self, input:Tensor, target:Tensor) -> Tensor:
        err = input - target
        Ferr = torch.fft.fftn(err, norm=self.norm, dim=self.dim)
        # F_in = torch.fft.fftn(input, norm=self.norm, dim=self.dim)
        F_tar = torch.fft.fftn(target, norm=self.norm, dim=self.dim)

        loss = torch.abs(Ferr)**2
        weight = torch.exp(loss / (torch.abs(F_tar)**2 + 1e-8))
        weight = (weight ) / weight.mean()
        loss = loss * weight
        

        dim = [-(len(self.dim)+1)] + list(self.dim)
        loss = loss.mean(dim=dim)

        if self.reduction == 'mean': loss = loss.mean()
        elif self.reduction =='sum': loss = loss.sum()

        return loss
    
class CustomLoss(BaseLoss):
    def __init__(self, 
        isRelative:bool=False, 
        norm:Literal['forward', 'ortho', 'backward']='forward', 
        dim:Union[int, Iterable]=(-2, -1), 
        *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss1 = FourierCorrLoss(isRelative=isRelative, norm=norm, dim=dim)
        self.loss2 = nn.MSELoss() # FourierWeightedMSELoss(*args, **kwargs) # nn.MSELoss()
    
    def forward(self, input:Tensor, target:Tensor) -> Tensor:
        # loss1 = self.loss1(input, target)
        loss2 = self.loss2(input, target)
        loss = loss2#  + loss2
        return loss

test_criterion={
    'l2': nn.MSELoss(), 
    'h1': H1Loss(d=2, reduction='mean'), 
    'h2': HsLoss(k=2, reduction='mean'), # HsLoss(k=2, group=False, size_average=True), 
    'fftcorr': FourierCorrLoss(reduction='mean', isRelative=True,dim=(-2, -1), norm='backward'),
    
    'fRMS_k<8': BSMSE(kmax=8, dim=(-2, -1), mode='spectral', isRelative=True),
    'fRMS_8<k<16': BSMSE(kmin=8, kmax=16, dim=(-2, -1), mode='spectral', isRelative=True),
    'fRMS_k>16': BSMSE(kmin=16, dim=(-2, -1), mode='spectral', isRelative=True),
    'fRMS_k<kmax': BSMSE(kmax=16, dim=(-2, -1), mode='spectral', isRelative=True),
    'fRMS_k>kmax': BSMSE(kmin=16, dim=(-2, -1), mode='spectral', isRelative=True),
    'fRMS_k>train': BSMSE(kmin=32, dim=(-2, -1), mode='spectral', isRelative=True),
    'fRMS_k<train': BSMSE(kmax=32, dim=(-2, -1), mode='spectral', isRelative=True),

    'vor_rms': RMSLoss(dim=2, isRelative=True),
    # 'tke': TKELoss(dim=2, isRelative=True), 
    # 'dissipation': DissipationLoss(nu=nu, dim=2, isRelative=True), 
    # 'R_lambda': RelambdaLoss(nu=nu, dim=2, isRelative=True), 
    'R_squared': R2(), 
    }

##### setup dataset #####
dataloaders = {'train':[], 'valid': [], 'test': []}
for i, data_config in enumerate(config['data']):
    transform = CustomTransform()
    normalization = Standardize(normalization_path=data_config['normalization']['normalization_path'])

    idx_leadtime = data_config['idx_leadtime'] if 'idx_leadtime' in data_config else int(data_config['leadtime'] * integral_timescales[nu] / dt) 
    # load_data = data_config['stage'] in ['fit', 'train']
    dm = CustomDataModule(**data_config['params'], 
        n_stride=idx_leadtime, 
        # Ndata_train=500, # 500, 
        # Ndata_val=50, 
        # Ndata_test=50,
        # transform=transform,
        normalization=normalization,
        # load_data=load_data,
        )
    stage = data_config['stage']
    dm.setup(stage=stage)
    if stage in ['fit', 'valid']:
        dataloaders['valid'].append(dm.val_dataloader())
    if stage in ['fit', 'train']:
        dataloaders['train'].append(dm.train_dataloader())
    if stage in ['test']:
        dataloaders['test'].append(dm.test_dataloader())


Loaded dataset metadata: dict_keys(['nu', 'Lx', 'Ly', 'dx', 'dy', 'Nx', 'Ny', 'x', 'y', 'dt', 'Nt', 'T', 't0', 't'])
Data Loaded fromD:\RESEARCH\2DIso\Data\nu=0.00025_n=256_fDNS=64/train\2dHIT_nu=0.000225_n=256_T=14.5_fDNS=64_0.hdf5. shape: (50, 400, 1, 64, 64), memory: 327.68 (MB)
Data Loaded fromD:\RESEARCH\2DIso\Data\nu=0.00025_n=256_fDNS=64/train\2dHIT_nu=0.000225_n=256_T=14.5_fDNS=64_1.hdf5. shape: (50, 400, 1, 64, 64), memory: 327.68 (MB)
Data Loaded fromD:\RESEARCH\2DIso\Data\nu=0.00025_n=256_fDNS=64/train\2dHIT_nu=0.000225_n=256_T=14.5_fDNS=64_2.hdf5. shape: (50, 400, 1, 64, 64), memory: 327.68 (MB)
Data Loaded fromD:\RESEARCH\2DIso\Data\nu=0.00025_n=256_fDNS=64/train\2dHIT_nu=0.000225_n=256_T=14.5_fDNS=64_3.hdf5. shape: (50, 400, 1, 64, 64), memory: 327.68 (MB)
Data Loaded fromD:\RESEARCH\2DIso\Data\nu=0.00025_n=256_fDNS=64/train\2dHIT_nu=0.000225_n=256_T=14.5_fDNS=64_4.hdf5. shape: (50, 400, 1, 64, 64), memory: 327.68 (MB)
Data Loaded fromD:\RESEARCH\2DIso\Data\nu=0.00025_n=2

In [12]:
# from utils.neuraloperator import FNO
from torchinfo import summary

model = FNO(**config['model']['params'])
pl_module = CustomModule(
        model=model, 
        optconfig=config['optimizer'], 
        )
batch_size = config['data'][0]['params']['batch_size']
print(pl_module.device)
summary(
        pl_module.model,
        input_size=(batch_size, 1, N, N),       # batch_size, in_channels, H, W
        col_names=("input_size", "output_size", "num_params", "trainable"),
        depth=5,                           # how many nested modules to show
        verbose=1
    )


cuda
Layer (type:depth-idx)                             Input Shape               Output Shape              Param #                   Trainable
FNO                                                [64, 1, 64, 64]           [64, 1, 64, 64]           --                        Partial
├─ChannelMLP: 1-1                                  [64, 1, 64, 64]           [64, 64, 64, 64]          8,256                     True
│    └─ModuleList: 2-3                             --                        --                        (recursive)               True
│    │    └─Conv1d: 3-1                            [64, 1, 4096]             [64, 128, 4096]           256                       True
├─ChannelMLP: 1-14                                 --                        --                        (recursive)               True
│    └─GELU: 2-2                                   [64, 128, 4096]           [64, 128, 4096]           --                        --
├─ChannelMLP: 1-3                                  

Layer (type:depth-idx)                             Input Shape               Output Shape              Param #                   Trainable
FNO                                                [64, 1, 64, 64]           [64, 1, 64, 64]           --                        Partial
├─ChannelMLP: 1-1                                  [64, 1, 64, 64]           [64, 64, 64, 64]          8,256                     True
│    └─ModuleList: 2-3                             --                        --                        (recursive)               True
│    │    └─Conv1d: 3-1                            [64, 1, 4096]             [64, 128, 4096]           256                       True
├─ChannelMLP: 1-14                                 --                        --                        (recursive)               True
│    └─GELU: 2-2                                   [64, 128, 4096]           [64, 128, 4096]           --                        --
├─ChannelMLP: 1-3                                  --   

In [13]:
from utils.Trainer import Trainer
from utils.DLCallbacks import CheckPoint, EarlyStopping
from utils.CFDCallback_Spectrum import CFDCallback_Spectrum
from utils.CFDCallback_PhaseError import CFDCallback_PhaseError
from utils.CFDCallback_PDF import CFDCallback_PDF
from utils.CFDCallback_Field import CFDCallback_Field

def main(device='cuda'):
    logger = None 
    logger = wandb.init(project=config['name'], 
                        # name=config['name'], 
                        config=config
                        )
    if 'seed' in config: torch.manual_seed(config['seed'])

    data_config = config['data'][0]
    fname=f"2dHIT_{config['model']['name']}_nu={data_config['nu']}"
    fname += f"_T={data_config['idx_leadtime']}" if 'idx_leadtime' in data_config else f"_T={data_config['leadtime']}TL"
    if logger: fname += f'_{logger.name}'

    optconfig = config['optimizer']
    if 'loss_fn' in config: optconfig['loss_fn'] = config['loss_fn']
    if 'lr' in config: optconfig['params']['lr'] = config['lr']
    modelconfig = {**config['model'], }
    modelconfig['params'].update({k: v for k, v in config.items() if k in config['model']['params']})
    
    model = FNO(**config['model']['params'])
    pl_module = CustomModule(
            model=model, 
            optconfig=config['optimizer'], 
            loss_fn=CustomLoss(reduction='mean', isRelative=True,dim=(-2, -1), norm='backward'), # test_criterion['h1'],
            fine_tunner = FineTunner,
            )
    # pl_module.load('./pretrain/2dHIT_FNO_nu=5e-05_T=0.1TL_different-dust-234_top0', 
    #     # load_opt=False, 
    #     # freeze=True, 
    #     )
    if logger: wandb.watch(pl_module, log="all", log_freq=100)
    callbacks = [
            # FineTunner(loss_fn=test_criterion['h1']),
            CFDCallback_Spectrum(file_dir='./result/', fname=fname), 
            CFDCallback_PhaseError(file_dir='./result/', fname=fname), 
            CFDCallback_PDF(file_dir='./result/', fname=fname), 
            CFDCallback_Field(file_dir='./result/', fname=fname), 
            CustomCallback(criterions=test_criterion, file_dir='./result/', fname=fname, device=device,
                        ),
            CheckPoint(
                ckpt_name=fname, ckpt_path='./checkpoint/', 
                every_n_epoch = 1,
                criterion=nn.MSELoss(), mode = 'min', 
                load_ckpt=False, 
                ),
            EarlyStopping(criterion=nn.MSELoss(), # neuralop.H1Loss(d=2, reductions='mean'),# 
                            mode='min', min_delta=1e-5, 
                            patience=5, verbose=True, divergence_threshold=1e3, 
                            stopping_threshold=1e-3, 
                            ),
            ]
    trainer = Trainer(
        max_epochs=config['epochs'],
        check_val_every_n_epochs=1,
        # check_val_every_n_iter=30000, 
        enable_progress_bar=True,
        callbacks=callbacks, 
        logger=logger,
        )
    
    trainer.fit(
        model=pl_module,
        train_dataloaders=dataloaders['train'], 
        val_dataloaders=dataloaders['valid'],
        # datamodule = datamodules,
        )
    if logger: wandb.finish()
    return 

In [14]:
if __name__ == "__main__":
    main()

wandb: Currently logged in as: hjungwon034 (jungwonheo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


  3%|▎         | 99/2851 [00:34<12:15,  3.74it/s] c:\Users\JungwonHeo\anaconda3\envs\FNOv1\lib\site-packages\wandb\integration\torch\wandb_torch.py:197: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Copy.cpp:250.)
  flat = flat.type(torch.cuda.FloatTensor)
100%|██████████| 2851/2851 [13:32<00:00,  3.51it/s]  
c:\Users\JungwonHeo\anaconda3\envs\FNOv1\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\JungwonHeo\anaconda3\envs\FNOv1\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 285/285 [00:52<00:00,  5.44it/s]


  Validation Epoch: 0, l2: 0.026258, h1: 0.261230, h2: 0.547523, fftcorr: 0.013221, fRMS_k<8: 0.003981, fRMS_8<k<16: 0.024596, fRMS_k>16: 0.156438, fRMS_k<kmax: 0.007430, fRMS_k>kmax: 0.156438, fRMS_k>train: 0.729609, fRMS_k<train: 0.021932, vor_rms: 0.014225, R_squared: 0.973705


100%|██████████| 5/5 [01:54<00:00, 22.93s/it]


  Validation Epoch: 0, l2: 0.192807, h1: 1.032330, h2: 1.832671, fftcorr: 0.102136, fRMS_k<8: 0.007103, fRMS_8<k<16: 0.077646, fRMS_k>16: 1.040778, fRMS_k<kmax: 0.020304, fRMS_k>kmax: 1.040778, fRMS_k>train: 1.414054, fRMS_k<train: 0.112965, vor_rms: 0.079555, R_squared: 0.814938


...

  Validation Epoch: 8, l2: 0.016439, h1: 0.205515, h2: 0.442982, fftcorr: 0.008251, fRMS_k<8: 0.002221, fRMS_8<k<16: 0.018812, fRMS_k>16: 0.092216, fRMS_k<kmax: 0.004999, fRMS_k>kmax: 0.092216, fRMS_k>train: 0.499853, fRMS_k<train: 0.013458, vor_rms: 0.007664, R_squared: 0.983544


100%|██████████| 5/5 [02:18<00:00, 27.63s/it]


  Validation Epoch: 8, l2: 0.184645, h1: 1.002216, h2: 1.550311, fftcorr: 0.097547, fRMS_k<8: 0.005890, fRMS_8<k<16: 0.078510, fRMS_k>16: 0.997069, fRMS_k<kmax: 0.019476, fRMS_k>kmax: 0.997069, fRMS_k>train: 1.324199, fRMS_k<train: 0.110035, vor_rms: 0.084199, R_squared: 0.822721


 57%|█████▋    | 1636/2851 [07:53<05:52,  3.45it/s]  


KeyboardInterrupt: 